# iMet 2020 FGVC7: Plan

Objectives:
- Get a strong baseline quickly; iterate to medal.
- Maintain rigorous CV and logging; avoid long blind runs.

Initial Baseline (Phase 1):
- Environment check: confirm GPU and correct torch stack.
- Data sanity check: train.csv, labels.csv, sample_submission.csv, image paths.
- CV: 5-fold Multilabel Stratified KFold (iterative stratification).
- Model: timm pretrained CNN (e.g., tf_efficientnet_b3_ns or nfnet_l0 if VRAM allows), multilabel BCEWithLogitsLoss.
- Image size 384→512 (start 384 for speed), AMP + gradient accumulation if needed.
- Augmentations: A.Resize->A.RandomResizedCrop(384), HFlip, ColorJitter(soft), Cutout optional; Normalize as timm pretrained.
- Optimizer: AdamW, cosine schedule with warmup. Early stopping on F1.
- Thresholding: per-fold global threshold via sweep maximizing micro-F1 on OOF; save per-fold thresholds.
- Artifacts: save OOF logits, test logits, model weights per fold. Log times/folds.

Improvements (Phase 2):
- Larger resolution (512), stronger aug (Mixup/Cutmix careful with multilabel), EMA, label-smoothing.
- Try different backbones (Swin-T/S, ConvNeXt-T, EfficientNetV2-S/B3).
- TTA (hflip + minor scale).
- Blend diverse seeds/backbones via logit averaging and re-threshold using OOF.

Validation Discipline:
- Single fold split saved and reused across runs.
- Avoid leakage: fit transforms inside folds only; no peeking.
- Track micro-F1 OOF; expect strong baseline ~0.60–0.63 at 384, improve to ≥0.65 with 512/backbone/ensemble.

Execution Plan:
1) Env check (GPU, torch install).
2) Data EDA: counts, classes, label freq, basic sanity.
3) Implement training pipeline script (train.py) with cfg and logging.
4) Smoke test on 1 fold, 1000 images, few epochs to verify.
5) Full 5-fold at 384. Save OOF/test logits.
6) Threshold sweep, create submission. Request expert review.
7) Iterate with improved backbone/resolution/ensembles until medal CV.

We will solicit expert advice at each major milestone.

In [2]:
import os, sys, subprocess, json, time, shutil, pandas as pd
from pathlib import Path

print('=== Environment Check ===', flush=True)
try:
    out = subprocess.run(['bash','-lc','nvidia-smi || true'], capture_output=True, text=True, check=False)
    print(out.stdout)
except Exception as e:
    print('nvidia-smi failed:', e)

try:
    import torch
    print('torch:', torch.__version__, 'CUDA avail:', torch.cuda.is_available())
    if torch.cuda.is_available():
        print('GPU name:', torch.cuda.get_device_name(0))
except Exception as e:
    print('torch not available yet:', e)

print('=== Data Sanity ===', flush=True)
base = Path('.')
train_dir = base/'train'
test_dir = base/'test'

# Count any files (any extension) to avoid extension mismatch issues
train_files = [p.name for p in train_dir.iterdir() if p.is_file()]
test_files = [p.name for p in test_dir.iterdir() if p.is_file()]
print('train files:', len(train_files))
print('test files:', len(test_files))
print('sample train files:', train_files[:5])
print('sample test files:', test_files[:5])

train_csv = pd.read_csv(base/'train.csv')
labels_csv = pd.read_csv(base/'labels.csv')
sub_csv = pd.read_csv(base/'sample_submission.csv')
print('train.csv shape:', train_csv.shape)
print('labels.csv shape:', labels_csv.shape)
print('sample_submission.csv shape:', sub_csv.shape)
print('train.csv head:\n', train_csv.head(3))
print('labels.csv head:\n', labels_csv.head(3))

# Determine extension by probing first few ids
def find_ext_for_id(img_id: str, roots):
    for ext in ('.jpg', '.jpeg', '.png', '.webp', '.bmp'):
        for root in roots:
            p = root/f'{img_id}{ext}'
            if p.exists():
                return ext
    # fallback: scan by prefix
    for root in roots:
        cands = list(root.glob(f'{img_id}.*'))
        if cands:
            return cands[0].suffix
    return None

train_id_col = 'id' if 'id' in train_csv.columns else ('image_id' if 'image_id' in train_csv.columns else None)
test_id_col = 'id' if 'id' in sub_csv.columns else ('image_id' if 'image_id' in sub_csv.columns else None)
print('Detected id columns -> train:', train_id_col, ' test:', test_id_col)

probe_ids = list(train_csv[train_id_col].head(5)) if train_id_col else []
probe_exts = {pid: find_ext_for_id(pid, [train_dir]) for pid in probe_ids}
print('Probe extensions:', probe_exts)
default_ext = None
vals = [e for e in probe_exts.values() if e]
if vals:
    default_ext = max(set(vals), key=vals.count)
print('Chosen default ext:', default_ext)

# Basic existence checks using detected extension or prefix matching
missing_train = 0
for img in train_csv[train_id_col].head(1000):
    if default_ext:
        exists = (train_dir/f'{img}{default_ext}').exists()
    else:
        exists = any((train_dir/f).name.startswith(img) for f in train_dir.iterdir())
    if not exists:
        missing_train += 1
print('Missing among first 1000 train ids:', missing_train)

missing_test = 0
for img in sub_csv[test_id_col].head(1000):
    if default_ext:
        exists = (test_dir/f'{img}{default_ext}').exists()
    else:
        exists = any((test_dir/f).name.startswith(img) for f in test_dir.iterdir())
    if not exists:
        missing_test += 1
print('Missing among first 1000 test ids:', missing_test)

print('Unique labels in labels.csv:', labels_csv['attribute_id'].nunique() if 'attribute_id' in labels_csv.columns else 'N/A')
print('Done.')

=== Environment Check ===


Sat Sep 27 16:44:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

train files: 120801
test files: 21318
sample train files: ['f4e684acbb6f2b33b45a16e586f87369.png', '727510fa873bed3a7a5b9902567b0d9f.png', 'e6b8484abfef0045ce9e4577f2ffd9e8.png', '8a074a477f1ccbd865151a9866419940.png', 'b22aa8832224499fdff061348f07bed2.png']
sample test files: ['4e0ba2b09affaf8525695752214b1dc4.png', 'ac5b7b0322f8c2ef4035b394956da403.png', 'df1d4aa72346aacaa4d4d1cc880fcc78.png', '878e96135e73845501059fb1d022d459.png', 'b007aba41e9d5a4c2df63fa452bf1640.png']
train.csv shape: (120801, 2)
labels.csv shape: (3474, 2)
sample_submission.csv shape: (21318, 2)
train.csv head:
                                  id                attribute_ids
0  4d0f6eada4ccb283551bc2f75e2ba588  3077 3187 3418 448 1625 782
1  75a9baea36b82e81263716fac427e416        2802 287 370 1419 784
2  cc7cbf14ef9e9261508ba27f9d2f4f28                      922 785
labels.csv head:
    attribute_id        attribute_name
0             0  country::afghanistan
1             1     country::alamania
2             2

In [3]:
import os, sys, subprocess, shutil
from pathlib import Path

def pip(*args):
    print('> pip', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

print('=== Install CUDA 12.1 torch stack and deps ===', flush=True)
# Uninstall any existing torch stack
for pkg in ('torch','torchvision','torchaudio'):
    try:
        subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    except Exception as e:
        print('uninstall error:', pkg, e)

# Clean potential stray site dirs that can shadow correct wheels
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

# Install exact cu121 torch stack
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# Freeze versions for later installs
Path('constraints.txt').write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')

# Core deps
pip('install', '-c', 'constraints.txt',
    'timm==1.0.9',
    'albumentations==1.4.14',
    'scikit-learn==1.5.2',
    'iterative-stratification==0.1.7',
    'opencv-python-headless==4.10.0.84',
    'pandas',
    '--upgrade-strategy', 'only-if-needed')

import torch
print('torch:', torch.__version__, 'CUDA build:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version, 'cuda', '12.1')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0))
print('Done installing.')

=== Install CUDA 12.1 torch stack and deps ===


> pip install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 392.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 382.9 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 287.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 161.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 428.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 449.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 426.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 483.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 428.5 MB/s eta 0:00:00


torch: 2.4.1+cu121 CUDA build: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q
Done installing.


In [4]:
# Create and save 5-fold Multilabel Stratified CV splits
import time, numpy as np, pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

t0 = time.time()
train_df = pd.read_csv('train.csv')
labels_df = pd.read_csv('labels.csv')
num_labels = labels_df['attribute_id'].nunique()
print('Num samples:', len(train_df), 'Num labels:', num_labels, flush=True)

# Parse attribute_ids -> list of ints
attrs = train_df['attribute_ids'].fillna('').apply(lambda s: [int(x) for x in str(s).split() if x!=''])
lens = attrs.apply(len)
print('Label cardinality: mean', lens.mean(), 'median', lens.median(), 'max', lens.max(), flush=True)

# Build dense indicator matrix (bool) for stratification
y = np.zeros((len(train_df), num_labels), dtype=np.uint8)
t1 = time.time()
for i, lab_list in enumerate(attrs):
    if lab_list:
        y[i, lab_list] = 1
    if (i+1) % 20000 == 0:
        print(f'..filled {i+1}/{len(train_df)} rows in {time.time()-t1:.1f}s', flush=True)

skf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = np.full(len(train_df), -1, dtype=np.int16)
for fold, (_, val_idx) in enumerate(skf.split(train_df.index.values, y)):
    folds[val_idx] = fold
    print(f'Assigned fold {fold}: {len(val_idx)} samples', flush=True)
assert (folds >= 0).all()

train_folds = train_df.copy()
train_folds['fold'] = folds
train_folds.to_csv('train_folds.csv', index=False)
print('Saved train_folds.csv. Time:', round(time.time()-t0,1), 's', flush=True)

Num samples: 120801 Num labels: 3474


Label cardinality: mean 4.421097507470964 median 4.0 max 26


..filled 20000/120801 rows in 0.0s


..filled 40000/120801 rows in 0.1s


..filled 60000/120801 rows in 0.1s


..filled 80000/120801 rows in 0.1s


..filled 100000/120801 rows in 0.2s


..filled 120000/120801 rows in 0.2s


KeyboardInterrupt: 

In [5]:
import sys, subprocess
def pip(*args):
    print('> pip', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

print('Fixing albumentations/albucore mismatch by downgrading albumentations to 1.3.1', flush=True)
pip('install', '-c', 'constraints.txt', 'albumentations==1.3.1', '--upgrade-strategy', 'only-if-needed')
import albumentations as A
print('albumentations version:', A.__version__)
from albumentations.pytorch import ToTensorV2
print('Albumentations import OK')

Fixing albumentations/albucore mismatch by downgrading albumentations to 1.3.1


> pip install -c constraints.txt albumentations==1.3.1 --upgrade-strategy only-if-needed


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 KB 6.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 208.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 218.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 246.7 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 292.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 KB 383.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 421.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 KB 528.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 561.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 253.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.1/230.1 KB 498.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 KB 516.2 MB/s eta 0:00:00


ImportError: cannot import name 'preserve_channel_dim' from 'albucore.utils' (/app/.pip-target/albucore/utils.py)

In [13]:
import sys, subprocess, importlib, os
def run_pip(cmd):
    print('> pip', *cmd, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *cmd], check=True)

print('Hard-reset albumentations to 1.3.1 (remove albucore), forcing overwrite', flush=True)
run_pip(['uninstall', '-y', 'albumentations', 'albucore'])
run_pip(['install', '--no-cache-dir', 'albumentations==1.3.1'])
import albumentations as A
print('albumentations version:', A.__version__)
print('albumentations file:', A.__file__)
from albumentations.pytorch import ToTensorV2
print('Albumentations import OK')

Hard-reset albumentations to 1.3.1 (remove albucore), forcing overwrite


> pip uninstall -y albumentations albucore


Found existing installation: albumentations 1.3.1
Uninstalling albumentations-1.3.1:
  Successfully uninstalled albumentations-1.3.1
> pip install --no-cache-dir albumentations==1.3.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 KB 5.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 169.0 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 176.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 208.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 KB 388.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 540.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 KB 511.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 426.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 542.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.1/230.1 KB 497.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 KB 529.1 MB/s eta 0:00:00


albumentations version: 1.3.1
albumentations file: /app/.pip-target/albumentations/__init__.py
Albumentations import OK


In [15]:
import os, sys, subprocess, time, shlex
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

print('=== Smoke: precompute fast folds (cardinality) + tiny 1-fold run (no-pretrained, unbuffered) ===', flush=True)
# 1) Fast folds via label-cardinality bins to avoid slow MSKF in subprocess
folds_path = Path('train_folds_smoke.csv')
if folds_path.exists():
    folds_path.unlink()
    print('Deleted existing train_folds_smoke.csv')
train_df = pd.read_csv('train.csv')
attrs = train_df['attribute_ids'].fillna('').astype(str).apply(lambda s: [int(x) for x in s.split() if x!=''])
card = attrs.apply(len).values
bins = np.clip(card, 0, 8)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = np.full(len(train_df), -1, dtype=np.int16)
for f, (_, vidx) in enumerate(skf.split(np.zeros(len(train_df)), bins)):
    folds[vidx] = f
train_df2 = train_df.copy()
train_df2['fold'] = folds
train_df2.to_csv(folds_path, index=False)
print('Wrote fast folds to train_folds_smoke.csv (cardinality SKFold)', flush=True)

# 2) Run a tiny smoke of train.py using these folds
cmd = [
    sys.executable, '-u', 'train.py',
    '--model', 'tf_efficientnet_b3_ns',
    '--img-size', '224',
    '--epochs', '1',
    '--batch-size', '64',
    '--val-batch-size', '96',
    '--num-workers', '4',
    '--folds', '0',
    '--folds-csv', 'train_folds_smoke.csv',
    '--out-dir', 'out_smoke_fast',
    '--limit-train-steps', '30',
    '--limit-val-steps', '10',
    '--early-stop-patience', '1',
    '--no-pretrained'
]
print('Running:', ' '.join(shlex.quote(x) for x in cmd), flush=True)
t0 = time.time()
env = dict(os.environ)
env['PYTHONUNBUFFERED'] = '1'
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, env=env)
try:
    for line in p.stdout:
        print(line, end='')
finally:
    rc = p.wait()
elapsed = time.time() - t0
print(f'Exit code: {rc}, elapsed {elapsed/60:.1f} min', flush=True)
assert rc == 0, 'Smoke training failed'
print('Smoke training completed.')

=== Smoke: precompute fast folds (cardinality) + tiny 1-fold run (no-pretrained, unbuffered) ===


Deleted existing train_folds_smoke.csv


Wrote fast folds to train_folds_smoke.csv (cardinality SKFold)


Running: /usr/bin/python3.11 -u train.py --model tf_efficientnet_b3_ns --img-size 224 --epochs 1 --batch-size 64 --val-batch-size 96 --num-workers 4 --folds 0 --folds-csv train_folds_smoke.csv --out-dir out_smoke_fast --limit-train-steps 30 --limit-val-steps 10 --early-stop-patience 1 --no-pretrained


Detected image extension: .png
==== Fold 0 start ====
/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b3_ns to current tf_efficientnet_b3.ns_jft_in1k.
  model = create_fn(


...

==== Fold 0 done: best_f1 0.17101 thr 0.090 ====


OOF micro-f1 0.08716 @ thr 0.050
/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:355: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.lo

Wrote submission.csv


Exit code: 0, elapsed 2.7 min


Smoke training completed.


In [18]:
import os, sys, subprocess, time, shlex
from pathlib import Path

print('=== PRODUCTION RUN: b3@384 5-fold, EMA+TTA (cardinality folds) ===', flush=True)
cmd = [
    sys.executable, 'train.py',
    '--model', 'tf_efficientnet_b3_ns',
    '--img-size', '384',
    '--epochs', '10',
    '--batch-size', '56',
    '--val-batch-size', '96',
    '--num-workers', '10',
    '--lr', '2e-4',
    '--use-ema',
    '--tta',
    '--early-stop-patience', '3',
    '--folds', '0,1,2,3,4',
    '--folds-csv', 'train_folds_smoke.csv',
    '--out-dir', 'out_b3_384_card',
    '--pretrained'
]
print('Running:', ' '.join(shlex.quote(x) for x in cmd), flush=True)
t0 = time.time()
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
try:
    for line in p.stdout:
        print(line, end='')
finally:
    rc = p.wait()
elapsed = time.time() - t0
print(f'Exit code: {rc}, elapsed {elapsed/3600:.2f} h', flush=True)
assert rc == 0, 'Production run failed'
print('Production run completed.')

=== PRODUCTION RUN: b3@384 5-fold, EMA+TTA (cardinality folds) ===


Running: /usr/bin/python3.11 train.py --model tf_efficientnet_b3_ns --img-size 384 --epochs 10 --batch-size 56 --val-batch-size 96 --num-workers 10 --lr 2e-4 --use-ema --tta --early-stop-patience 3 --folds 0,1,2,3,4 --folds-csv train_folds_smoke.csv --out-dir out_b3_384_card --pretrained


Detected image extension: .png
==== Fold 0 start ====
/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b3_ns to current tf_efficientnet_b3.ns_jft_in1k.
  model = create_fn(


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:215: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:244: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


...

fold 1 epoch 4 iter 1700/1725 loss 2.4279 elapsed 39.3m



=== VAL DIAG fold 1 epoch 4 ===
val_size=24160 probs_shape=(24160, 3474) tgts_shape=(24160, 3474)
probs_range=[0.000001,0.964950]
tgt_pos_rate=0.00127227 mean_pos_per_img=4.420


thr=0.2 pred_pos_rate=0.00719632 mean_pred_per_img=25.000 empty_frac=0.000000 TP=48827 FP=555173 FN=57957 f1@0.2=0.137389


fold 1 epoch 4 val micro-f1 0.13739 @ thr 0.050
Early stopping at epoch 4


==== Fold 1 done: best_f1 0.18269 thr 0.250 ====
==== Fold 2 start ====


In [17]:
# Precompute and lock 5-fold MSKF folds (seed=42) to train_folds.csv
import time, numpy as np, pandas as pd
from pathlib import Path
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from scipy.sparse import csr_matrix

t0 = time.time()
train_df = pd.read_csv('train.csv')
labels_df = pd.read_csv('labels.csv')
attr_ids = sorted(labels_df['attribute_id'].unique().tolist())
attr_to_idx = {a:i for i,a in enumerate(attr_ids)}
n = len(train_df); C = len(attr_ids)
print(f'n={n} classes={C}', flush=True)

# Build sparse label matrix via rows/cols
rows, cols = [], []
for i, s in enumerate(train_df['attribute_ids'].fillna('').astype(str).tolist()):
    if s:
        for a in map(int, s.split()):
            j = attr_to_idx.get(a, None)
            if j is not None:
                rows.append(i); cols.append(j)
print(f'nonzeros={len(rows)}', flush=True)
y_sparse = csr_matrix((np.ones(len(rows), dtype=np.uint8), (rows, cols)), shape=(n, C), dtype=np.uint8)

# Dense conversion (fits in RAM ~420MB) for iterstrat; one-time cost
t1 = time.time()
y = y_sparse.toarray(order='C')
print(f'dense built in {time.time()-t1:.1f}s, shape={y.shape}, dtype={y.dtype}', flush=True)

mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = np.full(n, -1, np.int16)
t2 = time.time()
for f, (_, vidx) in enumerate(mskf.split(np.zeros(n), y)):
    folds[vidx] = f
    print(f'fold {f}: {len(vidx)}', flush=True)
print(f'MSKF split time: {time.time()-t2:.1f}s', flush=True)
assert (folds >= 0).all()

out = train_df.copy(); out['fold'] = folds
out.to_csv('train_folds.csv', index=False)
print(f'Saved train_folds.csv in {time.time()-t0:.1f}s')

n=120801 classes=3474


nonzeros=534073


dense built in 0.0s, shape=(120801, 3474), dtype=uint8


KeyboardInterrupt: 

In [19]:
import os, sys, subprocess, time, shlex
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

print('=== Diagnostic single-fold run: b3@384, epochs=5, pretrained ===', flush=True)
# Ensure fast cardinality folds exist (for speed)
folds_path = Path('train_folds_smoke.csv')
if not folds_path.exists():
    train_df = pd.read_csv('train.csv')
    attrs = train_df['attribute_ids'].fillna('').astype(str).apply(lambda s: [int(x) for x in s.split() if x!=''])
    card = attrs.apply(len).values
    bins = np.clip(card, 0, 8)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    folds = np.full(len(train_df), -1, dtype=np.int16)
    for f, (_, vidx) in enumerate(skf.split(np.zeros(len(train_df)), bins)):
        folds[vidx] = f
    train_df2 = train_df.copy()
    train_df2['fold'] = folds
    train_df2.to_csv(folds_path, index=False)
    print('Wrote train_folds_smoke.csv')
else:
    print('Using existing train_folds_smoke.csv')

cmd = [
    sys.executable, '-u', 'train.py',
    '--model', 'tf_efficientnet_b3_ns',
    '--img-size', '384',
    '--epochs', '5',
    '--batch-size', '32',
    '--val-batch-size', '64',
    '--num-workers', '8',
    '--folds', '0',
    '--folds-csv', 'train_folds_smoke.csv',
    '--out-dir', 'out_debug',
    '--early-stop-patience', '2',
    '--pretrained'
]
print('Running:', ' '.join(shlex.quote(x) for x in cmd), flush=True)
t0 = time.time()
env = dict(os.environ)
env['PYTHONUNBUFFERED'] = '1'
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, env=env)
try:
    for line in p.stdout:
        print(line, end='')
finally:
    rc = p.wait()
elapsed = time.time() - t0
print(f'Exit code: {rc}, elapsed {elapsed/60:.1f} min', flush=True)
assert rc == 0, 'Diagnostic run failed'
print('Diagnostic run completed.')

=== Diagnostic single-fold run: b3@384, epochs=5, pretrained ===


Using existing train_folds_smoke.csv
Running: /usr/bin/python3.11 -u train.py --model tf_efficientnet_b3_ns --img-size 384 --epochs 5 --batch-size 32 --val-batch-size 64 --num-workers 8 --folds 0 --folds-csv train_folds_smoke.csv --out-dir out_debug --early-stop-patience 2 --pretrained


Detected image extension: .png
==== Fold 0 start ====
/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b3_ns to current tf_efficientnet_b3.ns_jft_in1k.
  model = create_fn(


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:214: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:246: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


...

fold 0 epoch 2 iter 700/3020 loss 0.0007 elapsed 12.4m


fold 0 epoch 2 iter 800/3020 loss 0.0007 elapsed 12.7m


fold 0 epoch 2 iter 900/3020 loss 0.0007 elapsed 13.0m


fold 0 epoch 2 iter 1000/3020 loss 0.0007 elapsed 13.3m


fold 0 epoch 2 iter 1100/3020 loss 0.0007 elapsed 13.5m


In [20]:
import os, sys, time, shlex, subprocess
from pathlib import Path
import numpy as np
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

print('=== Build fast MSKF folds: top-512 labels + cardinality bins (one-hot) ===', flush=True)
t0 = time.time()
train_df = pd.read_csv('train.csv')
labels_df = pd.read_csv('labels.csv')
attr_ids = labels_df['attribute_id'].astype(int).tolist()
n = len(train_df)
print('n samples:', n, 'num labels:', len(attr_ids), flush=True)

# Parse labels
labs = train_df['attribute_ids'].fillna('').astype(str).str.split()

# Count label frequencies
from collections import Counter
cnt = Counter()
for s in labs:
    for x in s:
        cnt[int(x)] += 1
topK = 512
top_attrs = [a for a,_ in cnt.most_common(topK)]
top_map = {a:i for i,a in enumerate(top_attrs)}

# Build reduced multilabel matrix: K (top labels) + 9 bin one-hots for cardinality (0..8, 8=8+)
K = len(top_attrs)
B = 9
y = np.zeros((n, K + B), dtype=np.uint8)
for i, s in enumerate(labs):
    if s:
        # top-K one-hots
        for x in s:
            j = top_map.get(int(x))
            if j is not None:
                y[i, j] = 1
        # cardinality bin one-hot
        c = len(s)
        b = min(c, B-1)
        y[i, K + b] = 1
    if (i+1) % 20000 == 0:
        print(f'..processed {i+1}/{n}', flush=True)

mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = np.full(n, -1, np.int16)
for f, (_, vidx) in enumerate(mskf.split(np.zeros(n), y)):
    folds[vidx] = f
    print('fold', f, 'size', len(vidx), flush=True)
assert (folds >= 0).all()

out_path = Path('train_folds_top512.csv')
out = train_df.copy(); out['fold'] = folds
out.to_csv(out_path, index=False)
print('Saved', str(out_path), 'in', f'{time.time()-t0:.1f}s', flush=True)

print('=== Launch 5-fold b3@384 EMA+TTA with new folds ===', flush=True)
cmd = [
    sys.executable, '-u', 'train.py',
    '--model', 'tf_efficientnet_b3_ns',
    '--img-size', '384',
    '--epochs', '10',
    '--batch-size', '56',
    '--val-batch-size', '96',
    '--num-workers', '10',
    '--lr', '2e-4',
    '--use-ema',
    '--tta',
    '--early-stop-patience', '3',
    '--folds', '0,1,2,3,4',
    '--folds-csv', 'train_folds_top512.csv',
    '--out-dir', 'out_b3_384_top512',
    '--pretrained'
]
print('Running:', ' '.join(shlex.quote(x) for x in cmd), flush=True)
t1 = time.time()
env = dict(os.environ); env['PYTHONUNBUFFERED'] = '1'
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, env=env)
try:
    for line in p.stdout:
        print(line, end='')
finally:
    rc = p.wait()
print(f'Exit code: {rc}, elapsed {(time.time()-t1)/3600:.2f} h', flush=True)
assert rc == 0, 'Production run failed'
print('Production run completed.')

=== Build fast MSKF folds: top-512 labels + cardinality bins (one-hot) ===


n samples: 120801 num labels: 3474


..processed 20000/120801


..processed 40000/120801


..processed 60000/120801


...

fold 2 epoch 4 iter 800/1725 loss 0.0005 elapsed 34.9m


fold 2 epoch 4 iter 900/1725 loss 0.0005 elapsed 35.4m


fold 2 epoch 4 iter 1000/1725 loss 0.0005 elapsed 36.0m


fold 2 epoch 4 iter 1100/1725 loss 0.0005 elapsed 36.5m


fold 2 epoch 4 iter 1200/1725 loss 0.0005 elapsed 37.0m


In [22]:
# Queue next production run with stabilized train.py (FP32 ASL + ColorJitter) at 448
import os, sys, time, shlex, subprocess
from pathlib import Path

print('=== Launch: b3@448 5-fold, EMA+TTA, using train_folds_top512.csv ===', flush=True)
assert Path('train_folds_top512.csv').exists(), 'Missing train_folds_top512.csv'
cmd = [
    sys.executable, '-u', 'train.py',
    '--model', 'tf_efficientnet_b3_ns',
    '--img-size', '448',
    '--epochs', '10',
    '--batch-size', '48',
    '--val-batch-size', '96',
    '--num-workers', '10',
    '--lr', '2e-4',
    '--use-ema',
    '--tta',
    '--early-stop-patience', '3',
    '--folds', '0,1,2,3,4',
    '--folds-csv', 'train_folds_top512.csv',
    '--out-dir', 'out_b3_448_top512',
    '--pretrained'
]
print('Running:', ' '.join(shlex.quote(x) for x in cmd), flush=True)
t0 = time.time()
env = dict(os.environ); env['PYTHONUNBUFFERED'] = '1'
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, env=env)
try:
    for line in p.stdout:
        print(line, end='')
finally:
    rc = p.wait()
print(f'Exit code: {rc}, elapsed {(time.time()-t0)/3600:.2f} h', flush=True)
assert rc == 0, 'b3@448 production run failed'
print('b3@448 production run completed.')

=== Launch: b3@448 5-fold, EMA+TTA, using train_folds_top512.csv ===


Running: /usr/bin/python3.11 -u train.py --model tf_efficientnet_b3_ns --img-size 448 --epochs 10 --batch-size 48 --val-batch-size 96 --num-workers 10 --lr 2e-4 --use-ema --tta --early-stop-patience 3 --folds 0,1,2,3,4 --folds-csv train_folds_top512.csv --out-dir out_b3_448_top512 --pretrained


Detected image extension: .png
==== Fold 0 start ====
/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b3_ns to current tf_efficientnet_b3.ns_jft_in1k.
  model = create_fn(


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:219: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:248: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


...


=== VAL DIAG fold 1 epoch 2 ===
val_size=24136 probs_shape=(24136, 3474) tgts_shape=(24136, 3474)
probs_range=[0.000067,0.998448]
tgt_pos_rate=0.00127339 mean_pos_per_img=4.424


thr=0.2 pred_pos_rate=0.02769274 mean_pred_per_img=96.205 empty_frac=0.000000 TP=102042 FP=2219952 FN=4730 f1@0.2=0.084028


fold 1 epoch 2 val micro-f1 0.58936 @ thr 0.500


fold 1 epoch 3 iter 100/2013 loss 0.0006 elapsed 28.2m


fold 1 epoch 3 iter 200/2013 loss 0.0006 elapsed 28.8m


In [25]:
import os, sys, time, shlex, subprocess
from pathlib import Path

print('=== Launch: b4@448 5-fold, EMA+TTA, using train_folds_top512.csv ===', flush=True)
assert Path('train_folds_top512.csv').exists(), 'Missing train_folds_top512.csv'
cmd = [
    sys.executable, '-u', 'train.py',
    '--model', 'tf_efficientnet_b4_ns',
    '--img-size', '448',
    '--epochs', '12',
    '--batch-size', '32',
    '--val-batch-size', '64',
    '--num-workers', '10',
    '--lr', '5e-4',
    '--use-ema',
    '--tta',
    '--early-stop-patience', '3',
    '--folds', '0,1,2,3,4',
    '--folds-csv', 'train_folds_top512.csv',
    '--out-dir', 'out_b4_448_top512',
    '--pretrained'
]
print('Running:', ' '.join(shlex.quote(x) for x in cmd), flush=True)
t0 = time.time()
env = dict(os.environ); env['PYTHONUNBUFFERED'] = '1'
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, env=env)
try:
    for line in p.stdout:
        print(line, end='')
finally:
    rc = p.wait()
print(f'Exit code: {rc}, elapsed {(time.time()-t0)/3600:.2f} h', flush=True)
assert rc == 0, 'b4@448 production run failed'
print('b4@448 production run completed.')

=== Launch: b4@448 5-fold, EMA+TTA, using train_folds_top512.csv ===


Running: /usr/bin/python3.11 -u train.py --model tf_efficientnet_b4_ns --img-size 448 --epochs 12 --batch-size 32 --val-batch-size 64 --num-workers 10 --lr 5e-4 --use-ema --tta --early-stop-patience 3 --folds 0,1,2,3,4 --folds-csv train_folds_top512.csv --out-dir out_b4_448_top512 --pretrained


Detected image extension: .png
==== Fold 0 start ====
/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:220: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:249: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


...

fold 0 epoch 9 iter 2200/3019 loss 0.0002 elapsed 158.2m


fold 0 epoch 9 iter 2300/3019 loss 0.0002 elapsed 158.7m


fold 0 epoch 9 iter 2400/3019 loss 0.0002 elapsed 159.2m


fold 0 epoch 9 iter 2500/3019 loss 0.0002 elapsed 159.8m


fold 0 epoch 9 iter 2600/3019 loss 0.0002 elapsed 160.3m


In [26]:
import os, sys, time, shlex, subprocess
from pathlib import Path

print('=== Prepared: convnext_tiny@384 5-fold, EMA+TTA, using train_folds_top512.csv ===', flush=True)
assert Path('train_folds_top512.csv').exists(), 'Missing train_folds_top512.csv'
cmd = [
    sys.executable, '-u', 'train.py',
    '--model', 'convnext_tiny_in22k',
    '--img-size', '384',
    '--epochs', '8',
    '--batch-size', '64',
    '--val-batch-size', '96',
    '--num-workers', '10',
    '--lr', '1.5e-4',
    '--use-ema',
    '--tta',
    '--early-stop-patience', '2',
    '--folds', '0,1,2,3,4',
    '--folds-csv', 'train_folds_top512.csv',
    '--out-dir', 'out_convnext_tiny_384_top512',
    '--pretrained'
]
print('Queued command (not running yet):', ' '.join(shlex.quote(x) for x in cmd), flush=True)

# To launch after current GPU job completes, run this cell to start the convnext_tiny baseline for ensembling.

=== Prepared: convnext_tiny@384 5-fold, EMA+TTA, using train_folds_top512.csv ===


Queued command (not running yet): /usr/bin/python3.11 -u train.py --model convnext_tiny_in22k --img-size 384 --epochs 8 --batch-size 64 --val-batch-size 96 --num-workers 10 --lr 1.5e-4 --use-ema --tta --early-stop-patience 2 --folds 0,1,2,3,4 --folds-csv train_folds_top512.csv --out-dir out_convnext_tiny_384_top512 --pretrained


In [28]:
import os, sys, time, shlex, subprocess
from pathlib import Path

print('=== Launch: convnext_tiny@384 5-fold, EMA+TTA, using train_folds_top512.csv (v2 settings) ===', flush=True)
assert Path('train_folds_top512.csv').exists(), 'Missing train_folds_top512.csv'
cmd = [
    sys.executable, '-u', 'train.py',
    '--model', 'convnext_tiny_in22k',
    '--img-size', '384',
    '--epochs', '10',
    '--batch-size', '64',
    '--val-batch-size', '96',
    '--num-workers', '10',
    '--lr', '2e-4',
    '--use-ema',
    '--tta',
    '--early-stop-patience', '3',
    '--folds', '0,1,2,3,4',
    '--folds-csv', 'train_folds_top512.csv',
    '--out-dir', 'out_convnext_tiny_384_top512_v2',
    '--pretrained'
]
print('Running:', ' '.join(shlex.quote(x) for x in cmd), flush=True)
t0 = time.time()
env = dict(os.environ); env['PYTHONUNBUFFERED'] = '1'
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, env=env)
try:
    for line in p.stdout:
        print(line, end='')
finally:
    rc = p.wait()
print(f'Exit code: {rc}, elapsed {(time.time()-t0)/3600:.2f} h', flush=True)
assert rc == 0, 'convnext_tiny@384 v2 production run failed'
print('convnext_tiny@384 v2 production run completed.')

=== Launch: convnext_tiny@384 5-fold, EMA+TTA, using train_folds_top512.csv (v2 settings) ===


Running: /usr/bin/python3.11 -u train.py --model convnext_tiny_in22k --img-size 384 --epochs 10 --batch-size 64 --val-batch-size 96 --num-workers 10 --lr 2e-4 --use-ema --tta --early-stop-patience 3 --folds 0,1,2,3,4 --folds-csv train_folds_top512.csv --out-dir out_convnext_tiny_384_top512_v2 --pretrained


Detected image extension: .png
==== Fold 0 start ====
/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name convnext_tiny_in22k to current convnext_tiny.fb_in22k.
  model = create_fn(


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:220: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:249: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


...

fold 0 epoch 2 val micro-f1 0.38873 @ thr 0.410


fold 0 epoch 3 iter 100/1509 loss 0.0005 elapsed 20.6m


fold 0 epoch 3 iter 200/1509 loss 0.0005 elapsed 21.2m


fold 0 epoch 3 iter 300/1509 loss 0.0005 elapsed 21.7m


fold 0 epoch 3 iter 400/1509 loss 0.0005 elapsed 22.3m


In [44]:
import os, sys, time, numpy as np, pandas as pd
from pathlib import Path

print('=== Blend setup: weighted logit averaging with global thr sweep or cardinality match ===', flush=True)

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def probs_to_logits(p, eps=1e-5):
    p = np.clip(p, eps, 1.0 - eps)
    return np.log(p / (1.0 - p))

def micro_f1_from_probs(probs, targets, thr=0.2):
    preds = (probs >= thr).astype(np.uint8)
    t = targets.astype(np.uint8)
    tp = np.logical_and(preds == 1, t == 1).sum(dtype=np.int64)
    fp = np.logical_and(preds == 1, t == 0).sum(dtype=np.int64)
    fn = np.logical_and(preds == 0, t == 1).sum(dtype=np.int64)
    denom = 2 * tp + fp + fn
    return float((2 * tp) / denom) if denom > 0 else 0.0

def build_y_true(train_csv='train.csv', labels_csv='labels.csv'):
    train_df = pd.read_csv(train_csv)
    labels_df = pd.read_csv(labels_csv)
    attr_ids = sorted(labels_df['attribute_id'].astype(int).unique().tolist())
    attr_to_idx = {a:i for i,a in enumerate(attr_ids)}
    y_true = np.zeros((len(train_df), len(attr_ids)), dtype=np.uint8)
    for i, s in enumerate(train_df['attribute_ids'].fillna('').astype(str)):
        if s:
            for a in map(int, s.split()):
                j = attr_to_idx.get(a, None)
                if j is not None:
                    y_true[i, j] = 1
    return train_df, np.array(attr_ids, dtype=np.int32), y_true

def load_model_artifacts(model_dir: Path):
    model_dir = Path(model_dir)
    oof_p = model_dir/'oof_probs.npy'
    test_p = model_dir/'test_probs.npy'
    meta_p = model_dir/'oof_meta.csv'
    oof = np.load(oof_p) if oof_p.exists() else None
    test = np.load(test_p) if test_p.exists() else None
    meta = pd.read_csv(meta_p) if meta_p.exists() else None
    return oof, test, meta

def blend_equal_weight(model_dirs, write_submission=True, out_name='submission_blend.csv', default_thr=0.50, cardinality_target=None, weights=None):
    model_dirs = [Path(d) for d in model_dirs]
    train_df, idx_to_attr, y_true = build_y_true('train.csv', 'labels.csv')
    train_mean_labels = float((y_true.sum(axis=1)).mean())
    # Load all artifacts
    models = []
    for d in model_dirs:
        oof, test, meta = load_model_artifacts(d)
        if test is None:
            print(f'[WARN] Missing test_probs.npy in {d}, skipping this model for test blend')
        models.append({'dir': d, 'oof': oof, 'test': test, 'meta': meta})

    have_test = [m for m in models if m['test'] is not None]
    if len(have_test) == 0:
        print('[INFO] No test outputs yet; cannot write submission.')
        return None, None, None

    # Prepare weights
    if weights is not None:
        if len(weights) != len(have_test):
            print('[WARN] Provided weights length does not match number of test models; ignoring weights.')
            weights_use = None
        else:
            weights_use = np.array(weights, dtype=np.float64)
    else:
        weights_use = None

    # Blend test in logit space
    test_logits_list = [probs_to_logits(m['test']) for m in have_test]
    if weights_use is None:
        Zt = np.mean(np.stack(test_logits_list, axis=0), axis=0)
    else:
        Zt = np.average(np.stack(test_logits_list, axis=0), axis=0, weights=weights_use)
    Pt = sigmoid(Zt)

    # Determine threshold
    have_oof = [m for m in models if m['oof'] is not None and m['meta'] is not None]
    oof_f1, best_thr = None, None
    if len(have_oof) > 0:
        oof_logits_list = [probs_to_logits(m['oof']) for m in have_oof]
        if weights_use is None or len(have_oof) != len(have_test):
            Zb = np.mean(np.stack(oof_logits_list, axis=0), axis=0)
        else:
            Zb = np.average(np.stack(oof_logits_list, axis=0), axis=0, weights=weights_use)
        Pb = sigmoid(Zb)
        thrs = np.arange(0.48, 0.5201, 0.002)
        f1s = [micro_f1_from_probs(Pb, y_true, thr=t) for t in thrs]
        bi = int(np.argmax(f1s))
        best_thr = float(thrs[bi])
        oof_f1 = float(f1s[bi])
        print(f'Blended OOF micro-f1 {oof_f1:.5f} @ thr {best_thr:.3f}')
    elif cardinality_target is not None:
        thrs = np.arange(0.48, 0.5201, 0.002)
        means = [float((Pt >= t).sum(axis=1).mean()) for t in thrs]
        target = float(cardinality_target)
        bi = int(np.argmin([abs(m - target) for m in means]))
        best_thr = float(thrs[bi])
        print(f'[CARD] Train mean labels/img={train_mean_labels:.3f}, target={target:.3f}, chosen thr={best_thr:.3f} (pred_mean={means[bi]:.3f})')
    else:
        best_thr = float(default_thr)
        print(f'[INFO] No OOF available; using default threshold {best_thr:.3f}')

    if write_submission and best_thr is not None:
        sub = pd.read_csv('sample_submission.csv')
        ids = sub['id'].values
        rows = []
        for i in range(len(ids)):
            p = Pt[i]
            pred_idx = np.where(p >= best_thr)[0].tolist()
            if len(pred_idx) == 0:
                pred_idx = [int(np.argmax(p))]
            pred_attr = [int(idx_to_attr[j]) for j in sorted(set(pred_idx))]
            rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(x) for x in pred_attr)})
        sub_df = pd.DataFrame(rows)
        sub_df.to_csv(out_name, index=False)
        print(f'Wrote {out_name} with thr={best_thr:.3f} using {len(have_test)} models')
    else:
        print('[INFO] Skipping submission write (best_thr not available).')
    return oof_f1, best_thr, Pt

# Example usage (will run later when artifacts exist):
MODEL_DIRS = [
    'out_b3_384_top512',
    'out_b3_448_top512',
    'out_convnext_tiny_384_top512',
]
print('Ready. Call blend_equal_weight(MODEL_DIRS, weights=[2,1,1], cardinality_target=4.42) to create weighted submission.', flush=True)

=== Blend setup: weighted logit averaging with global thr sweep or cardinality match ===


Ready. Call blend_equal_weight(MODEL_DIRS, weights=[2,1,1], cardinality_target=4.42) to create weighted submission.


In [33]:
import os, sys, glob, numpy as np, pandas as pd
from pathlib import Path
import train as trn

print('=== Inference-only helper: generate test_probs.npy from available fold weights ===', flush=True)

def infer_available_folds(out_dir: str, model_name: str, img_size: int, val_batch_size: int = 96, num_workers: int = 10, use_tta: bool = True):
    out = Path(out_dir)
    pths = sorted(out.glob(f'{model_name.replace("/","_")}_fold*.pth'))
    if len(pths) == 0:
        print(f'[SKIP] No weights found in {out_dir}')
        return None
    # Build minimal cfg object expected by train.infer_test
    class Cfg: pass
    cfg = Cfg()
    cfg.model = model_name
    cfg.img_size = img_size
    cfg.val_batch_size = val_batch_size
    cfg.num_workers = num_workers
    cfg.sample_sub = 'sample_submission.csv'
    cfg.test_dir = Path('test')
    cfg.use_ema = False
    cfg.tta = bool(use_tta)

    # Label mapping and image extension
    labels_df = pd.read_csv('labels.csv')
    attr_ids = sorted(labels_df['attribute_id'].astype(int).unique().tolist())
    attr_to_idx = {a:i for i,a in enumerate(attr_ids)}
    idx_to_attr = np.array(attr_ids, dtype=np.int32)
    train_df = pd.read_csv('train.csv')
    img_ext = trn.detect_ext(Path('train'), [train_df['id'].iloc[0]])

    print(f'[INFO] {out_dir}: found {len(pths)} fold weights -> running test inference (tta={cfg.tta})')
    ids, probs = trn.infer_test(cfg, [str(p) for p in pths], len(attr_ids), img_ext, attr_to_idx, idx_to_attr)
    np.save(out / 'test_probs.npy', probs)
    print(f'[DONE] Saved {out_dir}/test_probs.npy with shape {probs.shape}')
    return probs

# Example: run inference for partial dirs; re-run any time new weights appear
TRY_JOBS = [
    ('out_b3_384_top512', 'tf_efficientnet_b3_ns', 384),
    ('out_b3_448_top512', 'tf_efficientnet_b3_ns', 448),
    ('out_b3_384_card', 'tf_efficientnet_b3_ns', 384),
    ('out_convnext_tiny_384_top512', 'convnext_tiny_in22k', 384),
    ('out_convnext_tiny_384_top512_v2', 'convnext_tiny_in22k', 384),
]
for d, m, sz in TRY_JOBS:
    if Path(d).exists():
        try:
            infer_available_folds(d, m, sz, val_batch_size=96, num_workers=10, use_tta=True)
        except Exception as e:
            print(f'[WARN] Inference failed for {d}: {e}')
    else:
        print(f'[SKIP] Missing dir {d}')

print('Ready. You can re-run this cell any time new fold weights appear to refresh test_probs.npy for blending.', flush=True)

=== Inference-only helper: generate test_probs.npy from available fold weights ===


[INFO] out_b3_384_top512: found 2 fold weights -> running test inference (tta=True)


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:365: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(pth, map_location='cpu')
/app/

[DONE] Saved out_b3_384_top512/test_probs.npy with shape (21318, 3474)
[INFO] out_b3_448_top512: found 1 fold weights -> running test inference (tta=True)


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:365: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(pth, map_location='cpu')
/app/

[DONE] Saved out_b3_448_top512/test_probs.npy with shape (21318, 3474)
[INFO] out_b3_384_card: found 2 fold weights -> running test inference (tta=True)


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:365: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(pth, map_location='cpu')
/app/

[DONE] Saved out_b3_384_card/test_probs.npy with shape (21318, 3474)
[INFO] out_convnext_tiny_384_top512: found 1 fold weights -> running test inference (tta=True)


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:365: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(pth, map_location='cpu')
/app/

[DONE] Saved out_convnext_tiny_384_top512/test_probs.npy with shape (21318, 3474)
[SKIP] Missing dir out_convnext_tiny_384_top512_v2
Ready. You can re-run this cell any time new fold weights appear to refresh test_probs.npy for blending.


In [29]:
import os, sys, time, shlex, subprocess
from pathlib import Path

print('=== Launch: convnext_small@384 5-fold, EMA+TTA, using train_folds_top512.csv ===', flush=True)
assert Path('train_folds_top512.csv').exists(), 'Missing train_folds_top512.csv'
cmd = [
    sys.executable, '-u', 'train.py',
    '--model', 'convnext_small_in22k',
    '--img-size', '384',
    '--epochs', '10',
    '--batch-size', '56',
    '--val-batch-size', '96',
    '--num-workers', '10',
    '--lr', '2e-4',
    '--use-ema',
    '--tta',
    '--early-stop-patience', '3',
    '--folds', '0,1,2,3,4',
    '--folds-csv', 'train_folds_top512.csv',
    '--out-dir', 'out_convnext_small_384_top512',
    '--pretrained'
]
print('Running:', ' '.join(shlex.quote(x) for x in cmd), flush=True)
t0 = time.time()
env = dict(os.environ); env['PYTHONUNBUFFERED'] = '1'
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, env=env)
try:
    for line in p.stdout:
        print(line, end='')
finally:
    rc = p.wait()
print(f'Exit code: {rc}, elapsed {(time.time()-t0)/3600:.2f} h', flush=True)
assert rc == 0, 'convnext_small@384 production run failed'
print('convnext_small@384 production run completed.')

=== Launch: convnext_small@384 5-fold, EMA+TTA, using train_folds_top512.csv ===


Running: /usr/bin/python3.11 -u train.py --model convnext_small_in22k --img-size 384 --epochs 10 --batch-size 56 --val-batch-size 96 --num-workers 10 --lr 2e-4 --use-ema --tta --early-stop-patience 3 --folds 0,1,2,3,4 --folds-csv train_folds_top512.csv --out-dir out_convnext_small_384_top512 --pretrained


Detected image extension: .png
==== Fold 0 start ====
/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name convnext_small_in22k to current convnext_small.fb_in22k.
  model = create_fn(


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:220: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:249: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


...

fold 0 epoch 2 iter 1000/1725 loss 0.0006 elapsed 24.2m


fold 0 epoch 2 iter 1100/1725 loss 0.0006 elapsed 25.0m


fold 0 epoch 2 iter 1200/1725 loss 0.0006 elapsed 25.8m


fold 0 epoch 2 iter 1300/1725 loss 0.0006 elapsed 26.6m


fold 0 epoch 2 iter 1400/1725 loss 0.0006 elapsed 27.4m


In [34]:
print('=== Blending now: b3@384_top512 + b3@448_top512 + b3@384_card + convnext_tiny + OOF(threshold)=out_smoke_b3_384 ===', flush=True)
dirs = ['out_b3_384_top512','out_b3_448_top512','out_b3_384_card','out_convnext_tiny_384_top512','out_smoke_b3_384']
oof_f1, best_thr, Pt = blend_equal_weight(dirs, write_submission=True, out_name='submission.csv', default_thr=0.50)
print('Blend done. best_thr =', best_thr, 'OOF_f1 =', oof_f1)

=== Blending now: b3@384_top512 + b3@448_top512 + b3@384_card + convnext_tiny + OOF(threshold)=out_smoke_b3_384 ===


[WARN] Missing test_probs.npy in out_smoke_b3_384, skipping this model for test blend


Blended OOF micro-f1 0.08233 @ thr 0.050


Wrote submission.csv with thr=0.050 using 4 models
Blend done. best_thr = 0.05 OOF_f1 = 0.08233134629987264


In [35]:
print('=== Re-blend without OOF (use default thr=0.50) ===', flush=True)
dirs = ['out_b3_384_top512','out_b3_448_top512','out_b3_384_card','out_convnext_tiny_384_top512']
oof_f1, best_thr, Pt = blend_equal_weight(dirs, write_submission=True, out_name='submission.csv', default_thr=0.50)
print('Reblend done. best_thr =', best_thr, 'OOF_f1 =', oof_f1)

=== Re-blend without OOF (use default thr=0.50) ===


[INFO] No complete OOF across models; using default threshold 0.500.


Wrote submission.csv with thr=0.500 using 4 models
Reblend done. best_thr = 0.5 OOF_f1 = None


In [38]:
print('=== Re-blend with cardinality matching to train mean (~4.42) ===', flush=True)
dirs = ['out_b3_384_top512','out_b3_448_top512','out_b3_384_card','out_convnext_tiny_384_top512']
oof_f1, best_thr, Pt = blend_equal_weight(dirs, write_submission=True, out_name='submission.csv', default_thr=0.50, cardinality_target=4.42)
print('Reblend (cardinality) done. best_thr =', best_thr, 'OOF_f1 =', oof_f1)

=== Re-blend with cardinality matching to train mean (~4.42) ===


[CARD] Train mean labels/img=4.421, target=4.420, chosen thr=0.550 (pred_mean=4.559)
Wrote submission.csv with thr=0.550 using 4 models
Reblend (cardinality) done. best_thr = 0.5499999999999998 OOF_f1 = None


In [39]:
print('=== Safety blend: b3@384_top512 + b3@448_top512 + convnext_tiny | cardinality target=4.42 ===', flush=True)
dirs = ['out_b3_384_top512','out_b3_448_top512','out_convnext_tiny_384_top512']
oof_f1, best_thr, Pt = blend_equal_weight(dirs, write_submission=True, out_name='submission.csv', default_thr=0.50, cardinality_target=4.42)
print('Safety blend done. thr=', best_thr, 'OOF_f1=', oof_f1)

=== Safety blend: b3@384_top512 + b3@448_top512 + convnext_tiny | cardinality target=4.42 ===


[CARD] Train mean labels/img=4.421, target=4.420, chosen thr=0.495 (pred_mean=4.421)
Wrote submission.csv with thr=0.495 using 3 models
Safety blend done. thr= 0.4949999999999998 OOF_f1= None


In [40]:
import os, sys, time, shlex, subprocess
from pathlib import Path

print('=== Launch: b5@456 folds 0,1, EMA+TTA, using train_folds_top512.csv ===', flush=True)
assert Path('train_folds_top512.csv').exists(), 'Missing train_folds_top512.csv'
cmd = [
    sys.executable, '-u', 'train.py',
    '--model', 'tf_efficientnet_b5_ns',
    '--img-size', '456',
    '--epochs', '10',
    '--batch-size', '24',
    '--val-batch-size', '64',
    '--num-workers', '10',
    '--lr', '3e-4',
    '--use-ema',
    '--tta',
    '--early-stop-patience', '3',
    '--folds', '0,1',
    '--folds-csv', 'train_folds_top512.csv',
    '--out-dir', 'out_b5_456_top512',
    '--pretrained'
]
print('Running:', ' '.join(shlex.quote(x) for x in cmd), flush=True)
t0 = time.time()
env = dict(os.environ); env['PYTHONUNBUFFERED'] = '1'
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, env=env)
try:
    for line in p.stdout:
        print(line, end='')
finally:
    rc = p.wait()
print(f'Exit code: {rc}, elapsed {(time.time()-t0)/3600:.2f} h', flush=True)
assert rc == 0, 'b5@456 run failed'
print('b5@456 folds 0,1 completed.')

=== Launch: b5@456 folds 0,1, EMA+TTA, using train_folds_top512.csv ===


Running: /usr/bin/python3.11 -u train.py --model tf_efficientnet_b5_ns --img-size 456 --epochs 10 --batch-size 24 --val-batch-size 64 --num-workers 10 --lr 3e-4 --use-ema --tta --early-stop-patience 3 --folds 0,1 --folds-csv train_folds_top512.csv --out-dir out_b5_456_top512 --pretrained


Detected image extension: .png
==== Fold 0 start ====
/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b5_ns to current tf_efficientnet_b5.ns_jft_in1k.
  model = create_fn(


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:220: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:249: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


Traceback (most recent call last):
  File "/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py", line 484, in <module>
    main()
  File "/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py", line 445, in main
    val_idx, oof_probs, thr, f1 = train_one_fold(cfg, f, train_df, num_classes, img_ext, attr_to_idx)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py", line 250, in train_one_fold
    logits = model(imgs)
             ^^^^^^^^^^^
  File "/app/.pip-target/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/app/.pip-target/torch/nn/modules/module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/app/.pip-target/timm/models/efficientn

Exit code: 1, elapsed 0.02 h


AssertionError: b5@456 run failed

In [41]:
import os, sys, time, shlex, subprocess
from pathlib import Path

print('=== Relaunch: b5@456 fold 0 ONLY (reduced BS to avoid OOM) ===', flush=True)
assert Path('train_folds_top512.csv').exists(), 'Missing train_folds_top512.csv'
cmd = [
    sys.executable, '-u', 'train.py',
    '--model', 'tf_efficientnet_b5_ns',
    '--img-size', '456',
    '--epochs', '10',
    '--batch-size', '16',
    '--val-batch-size', '48',
    '--num-workers', '10',
    '--lr', '3e-4',
    '--use-ema',
    '--tta',
    '--early-stop-patience', '3',
    '--folds', '0',
    '--folds-csv', 'train_folds_top512.csv',
    '--out-dir', 'out_b5_456_top512',
    '--pretrained'
]
print('Running:', ' '.join(shlex.quote(x) for x in cmd), flush=True)
t0 = time.time()
env = dict(os.environ); env['PYTHONUNBUFFERED'] = '1'; env['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, env=env)
try:
    for line in p.stdout:
        print(line, end='')
finally:
    rc = p.wait()
print(f'Exit code: {rc}, elapsed {(time.time()-t0)/3600:.2f} h', flush=True)
assert rc == 0, 'b5@456 fold0 run failed'
print('b5@456 fold 0 completed.')

=== Relaunch: b5@456 fold 0 ONLY (reduced BS to avoid OOM) ===


Running: /usr/bin/python3.11 -u train.py --model tf_efficientnet_b5_ns --img-size 456 --epochs 10 --batch-size 16 --val-batch-size 48 --num-workers 10 --lr 3e-4 --use-ema --tta --early-stop-patience 3 --folds 0 --folds-csv train_folds_top512.csv --out-dir out_b5_456_top512 --pretrained


Detected image extension: .png
==== Fold 0 start ====
/app/.pip-target/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b5_ns to current tf_efficientnet_b5.ns_jft_in1k.
  model = create_fn(


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:220: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py:249: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


Traceback (most recent call last):
  File "/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py", line 484, in <module>
    main()
  File "/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py", line 445, in main
    val_idx, oof_probs, thr, f1 = train_one_fold(cfg, f, train_df, num_classes, img_ext, attr_to_idx)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/lib/simon/agent_run_states/imet-2020-fgvc7-20250927-162858/train.py", line 250, in train_one_fold
    logits = model(imgs)
             ^^^^^^^^^^^
  File "/app/.pip-target/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/app/.pip-target/torch/nn/modules/module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/app/.pip-target/timm/models/efficientn

In [42]:
print('=== Safety blend #2: b3@384_top512 + b3@448_top512 + convnext_tiny | cardinality target=4.33 ===', flush=True)
dirs = ['out_b3_384_top512','out_b3_448_top512','out_convnext_tiny_384_top512']
oof_f1, best_thr, Pt = blend_equal_weight(dirs, write_submission=True, out_name='submission_alt.csv', default_thr=0.50, cardinality_target=4.33)
print('Safety blend #2 done. thr=', best_thr, 'OOF_f1=', oof_f1)

=== Safety blend #2: b3@384_top512 + b3@448_top512 + convnext_tiny | cardinality target=4.33 ===


[CARD] Train mean labels/img=4.421, target=4.330, chosen thr=0.500 (pred_mean=4.332)
Wrote submission_alt.csv with thr=0.500 using 3 models
Safety blend #2 done. thr= 0.4999999999999998 OOF_f1= None


In [45]:
print('=== Weighted blends: 2:1:1 (b3_384:b3_448:convnext), prob clip->logits, narrow thr sweep via cardinality ===', flush=True)
dirs_main = ['out_b3_384_top512','out_b3_448_top512','out_convnext_tiny_384_top512']

# Main pick: match train mean ~4.42, overwrite submission.csv
oof_f1, best_thr, Pt = blend_equal_weight(dirs_main, weights=[2,1,1], write_submission=True, out_name='submission.csv', cardinality_target=4.42)
print('[MAIN] submission.csv thr=', best_thr, 'OOF_f1=', oof_f1)

# Backup: slightly lower cardinality target ~4.36
oof_f12, best_thr2, Pt2 = blend_equal_weight(dirs_main, weights=[2,1,1], write_submission=True, out_name='submission_weighted_alt.csv', cardinality_target=4.36)
print('[ALT ] submission_weighted_alt.csv thr=', best_thr2, 'OOF_f1=', oof_f12)

# Optional hedge: b3-only (drop convnext), 2:1 weight, at 4.42
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
oof_f1b3, thr_b3, _ = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=True, out_name='submission_b3_only.csv', cardinality_target=4.42)
print('[B3  ] submission_b3_only.csv thr=', thr_b3, 'OOF_f1=', oof_f1b3)
print('Weighted blending complete.')

=== Weighted blends: 2:1:1 (b3_384:b3_448:convnext), prob clip->logits, narrow thr sweep via cardinality ===


[CARD] Train mean labels/img=4.421, target=4.420, chosen thr=0.496 (pred_mean=4.403)
Wrote submission.csv with thr=0.496 using 3 models
[MAIN] submission.csv thr= 0.496 OOF_f1= None


[CARD] Train mean labels/img=4.421, target=4.360, chosen thr=0.498 (pred_mean=4.367)
Wrote submission_weighted_alt.csv with thr=0.498 using 3 models
[ALT ] submission_weighted_alt.csv thr= 0.498 OOF_f1= None


[CARD] Train mean labels/img=4.421, target=4.420, chosen thr=0.502 (pred_mean=4.406)
Wrote submission_b3_only.csv with thr=0.502 using 2 models
[B3  ] submission_b3_only.csv thr= 0.502 OOF_f1= None
Weighted blending complete.


In [46]:
import numpy as np, pandas as pd
from pathlib import Path

print('=== Post-process: per-group caps on weighted blend (2:1:1) ===', flush=True)

# 1) Recompute weighted blend probs (do not write submission here)
dirs_main = ['out_b3_384_top512','out_b3_448_top512','out_convnext_tiny_384_top512']
oof_f1, best_thr, Pt = blend_equal_weight(dirs_main, weights=[2,1,1], write_submission=False, out_name='noop.csv', cardinality_target=4.42)
print('[BLEND] thr chosen =', best_thr, 'OOF_f1=', oof_f1)

# 2) Build group mapping from labels.csv prefixes
labels_df = pd.read_csv('labels.csv')
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
attr_ids = labels_df['attribute_id'].astype(int).tolist()
attr_to_idx = {a:i for i,a in enumerate(sorted(attr_ids))}
idx_to_attr = np.array(sorted(attr_ids), dtype=np.int32)
idx_to_group = labels_df.set_index('attribute_id').loc[idx_to_attr, 'group'].values

# Inspect available groups
groups = pd.Series(idx_to_group).value_counts().index.tolist()
print('[GROUPS]', groups)

# 3) Define caps per group (fallback default=5)
caps = {
    'country': 1,
    'culture': 1,
    'century': 1,
    'object_type': 1,
    'technique': 2,
    'material': 2,
    'color': 1,
    'tag': 5,
    'tags': 5,
    'subject': 5,
}
default_cap = 5

# 4) Apply threshold then per-group caps
sub = pd.read_csv('sample_submission.csv')
ids = sub['id'].values
rows = []
thr = float(best_thr if best_thr is not None else 0.5)
for i in range(len(ids)):
    p = Pt[i]
    cand = np.where(p >= thr)[0]
    if cand.size == 0:
        # ensure at least one label
        top1 = int(np.argmax(p)); cand = np.array([top1], dtype=np.int64)
    # group -> indices within cand
    kept = []
    # sort candidates by prob desc
    cand_sorted = cand[np.argsort(-p[cand])]
    used_per_group = {}
    for j in cand_sorted:
        g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
        kcap = caps.get(g, default_cap)
        c = used_per_group.get(g, 0)
        if c < kcap:
            kept.append(j); used_per_group[g] = c + 1
    if len(kept) == 0:
        kept = [int(cand_sorted[0])]
    pred_attr = [int(idx_to_attr[j]) for j in kept]
    rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(x) for x in pred_attr)})

sub_caps = pd.DataFrame(rows)
sub_caps.to_csv('submission_caps.csv', index=False)
print('Wrote submission_caps.csv with per-group caps. thr=', thr)

=== Post-process: per-group caps on weighted blend (2:1:1) ===


[CARD] Train mean labels/img=4.421, target=4.420, chosen thr=0.496 (pred_mean=4.403)
[INFO] Skipping submission write (best_thr not available).
[BLEND] thr chosen = 0.496 OOF_f1= None
[GROUPS] ['medium', 'tags', 'culture', 'country', 'dimension']


Wrote submission_caps.csv with per-group caps. thr= 0.496


In [47]:
import shutil, os
print('=== Set final submission to per-group capped blend ===', flush=True)
src, dst = 'submission_caps.csv', 'submission.csv'
assert os.path.exists(src), f'Missing {src}; run cell 26 first.'
shutil.copyfile(src, dst)
print(f'Copied {src} -> {dst}')

=== Set final submission to per-group capped blend ===


Copied submission_caps.csv -> submission.csv


In [48]:
import numpy as np, pandas as pd, os, shutil
from pathlib import Path

print('=== Post-process: b3-only (2:1) weighted blend with per-group caps ===', flush=True)

# 1) Recompute b3-only weighted blend probs (no write) with a slightly lower target (4.38)
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
oof_f1_b3, thr_b3, Pt_b3 = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.38)
print('[BLEND B3] thr chosen =', thr_b3, 'OOF_f1=', oof_f1_b3)

# 2) Group mapping
labels_df = pd.read_csv('labels.csv')
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
attr_ids = labels_df['attribute_id'].astype(int).tolist()
attr_to_idx = {a:i for i,a in enumerate(sorted(attr_ids))}
idx_to_attr = np.array(sorted(attr_ids), dtype=np.int32)
idx_to_group = labels_df.set_index('attribute_id').loc[idx_to_attr, 'group'].values

# 3) Caps per group
caps = {
    'country': 1,
    'culture': 1,
    'century': 1,
    'object_type': 1,
    'technique': 2,
    'material': 2,
    'color': 1,
    'tag': 5,
    'tags': 5,
    'subject': 5,
}
default_cap = 5

# 4) Threshold then apply caps
sub = pd.read_csv('sample_submission.csv')
ids = sub['id'].values
rows = []
thr = float(thr_b3 if thr_b3 is not None else 0.5)
for i in range(len(ids)):
    p = Pt_b3[i]
    cand = np.where(p >= thr)[0]
    if cand.size == 0:
        top1 = int(np.argmax(p)); cand = np.array([top1], dtype=np.int64)
    cand_sorted = cand[np.argsort(-p[cand])]
    kept, used_per_group = [], {}
    for j in cand_sorted:
        g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
        kcap = caps.get(g, default_cap)
        c = used_per_group.get(g, 0)
        if c < kcap:
            kept.append(j); used_per_group[g] = c + 1
    if len(kept) == 0:
        kept = [int(cand_sorted[0])]
    pred_attr = [int(idx_to_attr[j]) for j in kept]
    rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(x) for x in pred_attr)})

sub_b3_caps = pd.DataFrame(rows)
sub_b3_caps.to_csv('submission_b3_caps.csv', index=False)
print('Wrote submission_b3_caps.csv with per-group caps. thr=', thr)

# 5) Make it the active submission for scoring
shutil.copyfile('submission_b3_caps.csv', 'submission.csv')
print('Copied submission_b3_caps.csv -> submission.csv')

=== Post-process: b3-only (2:1) weighted blend with per-group caps ===


[CARD] Train mean labels/img=4.421, target=4.380, chosen thr=0.504 (pred_mean=4.370)
[INFO] Skipping submission write (best_thr not available).
[BLEND B3] thr chosen = 0.504 OOF_f1= None


Wrote submission_b3_caps.csv with per-group caps. thr= 0.504
Copied submission_b3_caps.csv -> submission.csv


In [49]:
import numpy as np, pandas as pd, shutil, os
from pathlib import Path

print('=== Per-group caps v2 tuned to observed groups (medium, dimension, country, culture, tags) ===', flush=True)

def make_caps_submission(model_dirs, weights, target_card, out_path):
    # Blend to get probs
    oof_f1, best_thr, Pt = blend_equal_weight(model_dirs, weights=weights, write_submission=False, out_name='noop.csv', cardinality_target=target_card)
    print(f'[BLEND] thr={best_thr} target={target_card} models={model_dirs} weights={weights}')
    # Groups
    labels_df = pd.read_csv('labels.csv')
    labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
    attr_ids = labels_df['attribute_id'].astype(int).tolist()
    idx_to_attr = np.array(sorted(attr_ids), dtype=np.int32)
    idx_to_group = labels_df.set_index('attribute_id').loc[idx_to_attr, 'group'].values
    # Caps tuned to observed groups
    # Seen: ['medium','tags','culture','country','dimension']
    caps = {
        'country': 1,
        'culture': 1,
        'medium': 2,        # limit materials/technique-like to 2
        'dimension': 1,     # typically one dimension-related attr
        'tags': 5,
        'tag': 5,
    }
    default_cap = 3
    # Threshold + caps
    sub = pd.read_csv('sample_submission.csv')
    ids = sub['id'].values
    thr = float(best_thr if best_thr is not None else 0.5)
    rows = []
    for i in range(len(ids)):
        p = Pt[i]
        cand = np.where(p >= thr)[0]
        if cand.size == 0:
            cand = np.array([int(np.argmax(p))], dtype=np.int64)
        cand_sorted = cand[np.argsort(-p[cand])]
        used, kept = {}, []
        for j in cand_sorted:
            g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
            kcap = caps.get(g, default_cap)
            c = used.get(g, 0)
            if c < kcap:
                kept.append(j); used[g] = c + 1
        if len(kept) == 0:
            kept = [int(cand_sorted[0])]
        pred_attr = [int(idx_to_attr[j]) for j in kept]
        rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(x) for x in pred_attr)})
    sub_df = pd.DataFrame(rows)
    sub_df.to_csv(out_path, index=False)
    print('Wrote', out_path, 'thr=', thr)

# Produce two variants quickly
dirs_weighted = ['out_b3_384_top512','out_b3_448_top512','out_convnext_tiny_384_top512']
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']

make_caps_submission(dirs_weighted, weights=[2,1,1], target_card=4.42, out_path='submission_caps_v2_weighted.csv')
make_caps_submission(dirs_b3, weights=[2,1], target_card=4.38, out_path='submission_caps_v2_b3.csv')

# Default active: weighted v2
shutil.copyfile('submission_caps_v2_weighted.csv', 'submission.csv')
print('Set submission.csv -> submission_caps_v2_weighted.csv')

=== Per-group caps v2 tuned to observed groups (medium, dimension, country, culture, tags) ===


[CARD] Train mean labels/img=4.421, target=4.420, chosen thr=0.496 (pred_mean=4.403)
[INFO] Skipping submission write (best_thr not available).
[BLEND] thr=0.496 target=4.42 models=['out_b3_384_top512', 'out_b3_448_top512', 'out_convnext_tiny_384_top512'] weights=[2, 1, 1]


Wrote submission_caps_v2_weighted.csv thr= 0.496


[CARD] Train mean labels/img=4.421, target=4.380, chosen thr=0.504 (pred_mean=4.370)
[INFO] Skipping submission write (best_thr not available).
[BLEND] thr=0.504 target=4.38 models=['out_b3_384_top512', 'out_b3_448_top512'] weights=[2, 1]


Wrote submission_caps_v2_b3.csv thr= 0.504
Set submission.csv -> submission_caps_v2_weighted.csv


In [50]:
import numpy as np, pandas as pd, shutil
from pathlib import Path

print('=== B3-only v2 caps @ target=4.42 with post-cap thr sweep to mean~4.40 ===', flush=True)

# 1) Get b3-only blended probs (weights 2:1). We'll sweep thr post caps.
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
oof_f1_b3, thr_b3_base, Pt_b3 = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.42)
print('[BLEND B3] base thr=', thr_b3_base, 'OOF_f1=', oof_f1_b3)

# 2) Mapping and v2 caps tuned to observed groups
labels_df = pd.read_csv('labels.csv')
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
attr_ids = labels_df['attribute_id'].astype(int).tolist()
idx_to_attr = np.array(sorted(attr_ids), dtype=np.int32)
idx_to_group = labels_df.set_index('attribute_id').loc[idx_to_attr, 'group'].values
caps = {
    'country': 1,
    'culture': 1,
    'medium': 2,
    'dimension': 1,
    'tags': 5,
    'tag': 5,
}
default_cap = 3

sub = pd.read_csv('sample_submission.csv')
ids = sub['id'].values

def apply_caps(Pt, thr):
    rows = []
    counts = []
    for i in range(len(ids)):
        p = Pt[i]
        cand = np.where(p >= thr)[0]
        if cand.size == 0:
            cand = np.array([int(np.argmax(p))], dtype=np.int64)
        cand_sorted = cand[np.argsort(-p[cand])]
        used, kept = {}, []
        for j in cand_sorted:
            g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
            kcap = caps.get(g, default_cap)
            c = used.get(g, 0)
            if c < kcap:
                kept.append(j); used[g] = c + 1
        if len(kept) == 0:
            kept = [int(cand_sorted[0])]
        pred_attr = [int(idx_to_attr[j]) for j in kept]
        rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(x) for x in pred_attr)})
        counts.append(len(pred_attr))
    return rows, float(np.mean(counts))

# 3) Sweep global thr after caps to hit mean ~4.40
thrs = np.arange(0.494, 0.5061, 0.002)
target_mean = 4.40
best = None
best_rows = None
for t in thrs:
    rows_t, mean_t = apply_caps(Pt_b3, t)
    delta = abs(mean_t - target_mean)
    print(f'[SWEEP] thr={t:.3f} post-cap mean={mean_t:.3f} delta={delta:.3f}')
    if (best is None) or (delta < best[0]):
        best = (delta, t, mean_t)
        best_rows = rows_t

_, best_thr_cap, best_mean = best
print(f'[CHOSEN] thr={best_thr_cap:.3f} post-cap mean={best_mean:.3f}')
sub_df = pd.DataFrame(best_rows)
out_path = 'submission_b3_caps_442.csv'
sub_df.to_csv(out_path, index=False)
shutil.copyfile(out_path, 'submission.csv')
print('Wrote', out_path, 'and set as submission.csv')

=== B3-only v2 caps @ target=4.42 with post-cap thr sweep to mean~4.40 ===


[CARD] Train mean labels/img=4.421, target=4.420, chosen thr=0.502 (pred_mean=4.406)
[INFO] Skipping submission write (best_thr not available).
[BLEND B3] base thr= 0.502 OOF_f1= None


[SWEEP] thr=0.494 post-cap mean=4.000 delta=0.400
[SWEEP] thr=0.496 post-cap mean=3.975 delta=0.425


[SWEEP] thr=0.498 post-cap mean=3.952 delta=0.448
[SWEEP] thr=0.500 post-cap mean=3.929 delta=0.471


[SWEEP] thr=0.502 post-cap mean=3.903 delta=0.497
[SWEEP] thr=0.504 post-cap mean=3.880 delta=0.520


[SWEEP] thr=0.506 post-cap mean=3.857 delta=0.543
[CHOSEN] thr=0.494 post-cap mean=4.000
Wrote submission_b3_caps_442.csv and set as submission.csv


In [54]:
import numpy as np, pandas as pd, shutil
from pathlib import Path

print('=== B3-only v2 caps @ target=4.42 with wider post-cap thr sweep to hit mean~4.40 ===', flush=True)
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
oof_f1_b3, thr_b3_base, Pt_b3 = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.42)
print('[BLEND B3] base thr=', thr_b3_base, 'OOF_f1=', oof_f1_b3)

labels_df = pd.read_csv('labels.csv')
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
attr_ids = labels_df['attribute_id'].astype(int).tolist()
idx_to_attr = np.array(sorted(attr_ids), dtype=np.int32)
idx_to_group = labels_df.set_index('attribute_id').loc[idx_to_attr, 'group'].values
caps = {
    'country': 1,
    'culture': 1,
    'medium': 2,
    'dimension': 1,
    'tags': 5,
    'tag': 5,
}
default_cap = 3

sub = pd.read_csv('sample_submission.csv')
ids = sub['id'].values

def apply_caps(Pt, thr):
    rows = []; counts = []
    for i in range(len(ids)):
        p = Pt[i]
        cand = np.where(p >= thr)[0]
        if cand.size == 0:
            cand = np.array([int(np.argmax(p))], dtype=np.int64)
        cand_sorted = cand[np.argsort(-p[cand])]
        used, kept = {}, []
        for j in cand_sorted:
            g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
            kcap = caps.get(g, default_cap)
            c = used.get(g, 0)
            if c < kcap:
                kept.append(j); used[g] = c + 1
        if len(kept) == 0:
            kept = [int(cand_sorted[0])]
        pred_attr = [int(idx_to_attr[j]) for j in kept]
        rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(x) for x in pred_attr)})
        counts.append(len(pred_attr))
    return rows, float(np.mean(counts))

# Wider sweep to reach post-cap mean ~4.40
thrs = np.arange(0.460, 0.5201, 0.002)
target_mean = 4.40
best = None; best_rows = None
for t in thrs:
    rows_t, mean_t = apply_caps(Pt_b3, t)
    delta = abs(mean_t - target_mean)
    if (best is None) or (delta < best[0]):
        best = (delta, t, mean_t); best_rows = rows_t
    if int((t-0.460)/0.002) % 10 == 0:
        print(f'[SWEEP] thr={t:.3f} post-cap mean={mean_t:.3f}')

_, best_thr_cap, best_mean = best
print(f'[CHOSEN] thr={best_thr_cap:.3f} post-cap mean={best_mean:.3f}')
sub_df = pd.DataFrame(best_rows)
out_path = 'submission_b3_caps_442_tuned.csv'
sub_df.to_csv(out_path, index=False)
shutil.copyfile(out_path, 'submission.csv')
print('Wrote', out_path, 'and set as submission.csv')

=== B3-only v2 caps @ target=4.42 with wider post-cap thr sweep to hit mean~4.40 ===


[CARD] Train mean labels/img=4.421, target=4.420, chosen thr=0.502 (pred_mean=4.406)
[INFO] Skipping submission write (best_thr not available).
[BLEND B3] base thr= 0.502 OOF_f1= None


[SWEEP] thr=0.460 post-cap mean=4.436


[SWEEP] thr=0.480 post-cap mean=4.173


[SWEEP] thr=0.502 post-cap mean=3.903


[SWEEP] thr=0.520 post-cap mean=3.692
[CHOSEN] thr=0.462 post-cap mean=4.409
Wrote submission_b3_caps_442_tuned.csv and set as submission.csv


In [60]:
import numpy as np, pandas as pd, shutil
from pathlib import Path

print('=== B3-only v2 caps @ target=4.38 with wider post-cap thr sweep to hit mean~4.40 ===', flush=True)
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
oof_f1_b3, thr_b3_base, Pt_b3 = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.38)
print('[BLEND B3] base thr=', thr_b3_base, 'OOF_f1=', oof_f1_b3)

labels_df = pd.read_csv('labels.csv')
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
attr_ids = labels_df['attribute_id'].astype(int).tolist()
idx_to_attr = np.array(sorted(attr_ids), dtype=np.int32)
idx_to_group = labels_df.set_index('attribute_id').loc[idx_to_attr, 'group'].values
caps = {
    'country': 1,
    'culture': 1,
    'medium': 2,
    'dimension': 1,
    'tags': 5,
    'tag': 5,
}
default_cap = 3

sub = pd.read_csv('sample_submission.csv')
ids = sub['id'].values

def apply_caps(Pt, thr):
    rows = []; counts = []
    for i in range(len(ids)):
        p = Pt[i]
        cand = np.where(p >= thr)[0]
        if cand.size == 0:
            cand = np.array([int(np.argmax(p))], dtype=np.int64)
        cand_sorted = cand[np.argsort(-p[cand])]
        used, kept = {}, []
        for j in cand_sorted:
            g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
            kcap = caps.get(g, default_cap)
            c = used.get(g, 0)
            if c < kcap:
                kept.append(j); used[g] = c + 1
        if len(kept) == 0:
            kept = [int(cand_sorted[0])]
        pred_attr = [int(idx_to_attr[j]) for j in kept]
        rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(x) for x in pred_attr)})
        counts.append(len(pred_attr))
    return rows, float(np.mean(counts))

# Wider sweep to reach post-cap mean ~4.40
thrs = np.arange(0.460, 0.5201, 0.002)
target_mean = 4.40
best = None; best_rows = None
for t in thrs:
    rows_t, mean_t = apply_caps(Pt_b3, t)
    delta = abs(mean_t - target_mean)
    if (best is None) or (delta < best[0]):
        best = (delta, t, mean_t); best_rows = rows_t
    if int((t-0.460)/0.002) % 10 == 0:
        print(f'[SWEEP] thr={t:.3f} post-cap mean={mean_t:.3f}')

_, best_thr_cap, best_mean = best
print(f'[CHOSEN] thr={best_thr_cap:.3f} post-cap mean={best_mean:.3f}')
sub_df = pd.DataFrame(best_rows)
out_path = 'submission_b3_caps_438_tuned.csv'
sub_df.to_csv(out_path, index=False)
shutil.copyfile(out_path, 'submission.csv')
print('Wrote', out_path, 'and set as submission.csv')

=== B3-only v2 caps @ target=4.38 with wider post-cap thr sweep to hit mean~4.40 ===


[CARD] Train mean labels/img=4.421, target=4.380, chosen thr=0.504 (pred_mean=4.370)
[INFO] Skipping submission write (best_thr not available).
[BLEND B3] base thr= 0.504 OOF_f1= None


[SWEEP] thr=0.460 post-cap mean=4.436


[SWEEP] thr=0.480 post-cap mean=4.173


[SWEEP] thr=0.502 post-cap mean=3.903


[SWEEP] thr=0.520 post-cap mean=3.692
[CHOSEN] thr=0.462 post-cap mean=4.409
Wrote submission_b3_caps_438_tuned.csv and set as submission.csv


In [53]:
import numpy as np, pandas as pd, shutil
from pathlib import Path

print('=== B3-only v2 caps @ target=4.35 with wider post-cap thr sweep to hit mean~4.40 ===', flush=True)
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
oof_f1_b3, thr_b3_base, Pt_b3 = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.35)
print('[BLEND B3] base thr=', thr_b3_base, 'OOF_f1=', oof_f1_b3)

labels_df = pd.read_csv('labels.csv')
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
attr_ids = labels_df['attribute_id'].astype(int).tolist()
idx_to_attr = np.array(sorted(attr_ids), dtype=np.int32)
idx_to_group = labels_df.set_index('attribute_id').loc[idx_to_attr, 'group'].values
caps = {
    'country': 1,
    'culture': 1,
    'medium': 2,
    'dimension': 1,
    'tags': 5,
    'tag': 5,
}
default_cap = 3

sub = pd.read_csv('sample_submission.csv')
ids = sub['id'].values

def apply_caps(Pt, thr):
    rows = []; counts = []
    for i in range(len(ids)):
        p = Pt[i]
        cand = np.where(p >= thr)[0]
        if cand.size == 0:
            cand = np.array([int(np.argmax(p))], dtype=np.int64)
        cand_sorted = cand[np.argsort(-p[cand])]
        used, kept = {}, []
        for j in cand_sorted:
            g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
            kcap = caps.get(g, default_cap)
            c = used.get(g, 0)
            if c < kcap:
                kept.append(j); used[g] = c + 1
        if len(kept) == 0:
            kept = [int(cand_sorted[0])]
        pred_attr = [int(idx_to_attr[j]) for j in kept]
        rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(x) for x in pred_attr)})
        counts.append(len(pred_attr))
    return rows, float(np.mean(counts))

# Wider sweep to reach post-cap mean ~4.40
thrs = np.arange(0.460, 0.5201, 0.002)
target_mean = 4.40
best = None; best_rows = None
for t in thrs:
    rows_t, mean_t = apply_caps(Pt_b3, t)
    delta = abs(mean_t - target_mean)
    if (best is None) or (delta < best[0]):
        best = (delta, t, mean_t); best_rows = rows_t
    if int((t-0.460)/0.002) % 10 == 0:
        print(f'[SWEEP] thr={t:.3f} post-cap mean={mean_t:.3f}')

_, best_thr_cap, best_mean = best
print(f'[CHOSEN] thr={best_thr_cap:.3f} post-cap mean={best_mean:.3f}')
sub_df = pd.DataFrame(best_rows)
out_path = 'submission_b3_caps_435_tuned.csv'
sub_df.to_csv(out_path, index=False)
shutil.copyfile(out_path, 'submission.csv')
print('Wrote', out_path, 'and set as submission.csv')

=== B3-only v2 caps @ target=4.35 with wider post-cap thr sweep to hit mean~4.40 ===


[CARD] Train mean labels/img=4.421, target=4.350, chosen thr=0.506 (pred_mean=4.335)
[INFO] Skipping submission write (best_thr not available).
[BLEND B3] base thr= 0.506 OOF_f1= None


[SWEEP] thr=0.460 post-cap mean=4.436


[SWEEP] thr=0.502 post-cap mean=3.903


[SWEEP] thr=0.520 post-cap mean=3.692
[CHOSEN] thr=0.462 post-cap mean=4.409
Wrote submission_b3_caps_435_tuned.csv and set as submission.csv


In [56]:
import sys, subprocess, os, time, math, gc, numpy as np, pandas as pd
from pathlib import Path
from PIL import Image

print('=== Install and prepare CLIP kNN label transfer pipeline ===', flush=True)

# 0) Ensure deps (open_clip_torch, faiss-cpu) are installed
def pip(*args):
    print('> pip', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

try:
    import open_clip
except Exception:
    pip('install', 'open_clip_torch==2.26.1')
    import open_clip
try:
    import faiss
except Exception:
    pip('install', 'faiss-cpu==1.8.0.post1')
    import faiss

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

# 1) Dataset for CLIP embedding extraction
class ImgDataset(Dataset):
    def __init__(self, ids, root: Path, ext: str, transform):
        self.ids = ids
        self.root = Path(root)
        self.ext = ext
        self.t = transform
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, i):
        img_id = self.ids[i]
        p = self.root / f'{img_id}{self.ext}'
        with Image.open(p) as im:
            im = im.convert('RGB')
            x = self.t(im)
        return x, i

# 2) Extract CLIP embeddings for train/test and save
def extract_clip_embeddings(model_name='ViT-B-32', pretrained='laion2b_s34b_b79k', image_size=224, batch_size=256, num_workers=8, device='cuda'):
    import train as trn
    base = Path('.')
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('sample_submission.csv')
    # detect extension (png)
    img_ext = trn.detect_ext(Path('train'), [train_df['id'].iloc[0]])
    print('Detected ext:', img_ext, flush=True)

    # Load CLIP
    print('Loading CLIP:', model_name, pretrained, flush=True)
    model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained, device=device)
    model.eval()
    # Replace preprocess with minimal equivalent to keep control over size/normalize
    # open_clip preprocess already matches model's expected stats
    transform = preprocess

    def run_split(split_name, ids, img_root, out_path):
        ds = ImgDataset(ids, img_root, img_ext, transform)
        dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
        feats = np.zeros((len(ids), model.visual.output_dim), dtype=np.float32)
        t0 = time.time()
        seen = 0
        with torch.no_grad():
            for bi, (xb, idx) in enumerate(dl):
                xb = xb.to(device, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=True):
                    f = model.encode_image(xb)
                f = f.float()
                f = torch.nn.functional.normalize(f, dim=1)
                feats[idx.numpy()] = f.cpu().numpy()
                seen += xb.size(0)
                if (bi+1) % 20 == 0:
                    dt = time.time()-t0
                    ips = seen / max(dt,1e-6)
                    print(f'[{split_name}] {seen}/{len(ids)} in {dt/60:.1f}m ({ips:.1f} img/s)', flush=True)
        np.save(out_path, feats)
        print(f'[{split_name}] Saved {out_path} shape={feats.shape}', flush=True)
        del feats; gc.collect()

    # Train embeddings
    train_ids = train_df['id'].tolist()
    test_ids = test_df['id'].tolist()
    if not Path('clip_train_emb.npy').exists():
        run_split('train', train_ids, Path('train'), 'clip_train_emb.npy')
    else:
        print('[train] clip_train_emb.npy exists, skipping')
    if not Path('clip_test_emb.npy').exists():
        run_split('test', test_ids, Path('test'), 'clip_test_emb.npy')
    else:
        print('[test] clip_test_emb.npy exists, skipping')
    print('CLIP embedding extraction done.')

print('Ready: run this cell to install deps and define functions, then call extract_clip_embeddings() in the next cell.', flush=True)

=== Install and prepare CLIP kNN label transfer pipeline ===


> pip install open_clip_torch==2.26.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 416.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 KB 235.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 KB 280.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 KB 133.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 320.8 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 317.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 545.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 515.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 393.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 443.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 505.9 MB/s eta 0:00:00


ERROR: Operation cancelled by user


KeyboardInterrupt: 

In [57]:
import sys, subprocess, os, time, math, gc, numpy as np, pandas as pd
from pathlib import Path
from PIL import Image

print('=== Safe install for CLIP kNN (no torch upgrade) and define extract ===', flush=True)

def pip(*args):
    print('> pip', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# 1) Install open_clip without deps to avoid torch override; then install minimal deps under constraints
try:
    import open_clip
    print('open_clip already available')
except Exception:
    assert Path('constraints.txt').exists(), 'constraints.txt missing'
    pip('install', '--no-deps', 'open_clip_torch==2.26.1')
    # Minimal deps; honor torch constraints to prevent upgrades
    pip('install', '-c', 'constraints.txt', 'ftfy==6.3.1', 'tqdm', 'huggingface-hub==0.35.1', 'regex', 'safetensors', 'pillow', '--upgrade-strategy', 'only-if-needed')
    import open_clip  # noqa

try:
    import faiss  # noqa
    print('faiss available')
except Exception:
    pip('install', 'faiss-cpu==1.8.0.post1')
    import faiss  # noqa

import torch
from torch.utils.data import Dataset, DataLoader

class ImgDataset(Dataset):
    def __init__(self, ids, root: Path, ext: str, transform):
        self.ids = ids
        self.root = Path(root)
        self.ext = ext
        self.t = transform
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, i):
        img_id = self.ids[i]
        p = self.root / f'{img_id}{self.ext}'
        with Image.open(p) as im:
            im = im.convert('RGB')
            x = self.t(im)
        return x, i

def extract_clip_embeddings(model_name='ViT-B-32', pretrained='laion2b_s34b_b79k', batch_size=256, num_workers=8, device='cuda'):
    import open_clip
    import train as trn
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('sample_submission.csv')
    img_ext = trn.detect_ext(Path('train'), [train_df['id'].iloc[0]])
    print('Detected ext:', img_ext, flush=True)
    print('Loading CLIP:', model_name, pretrained, flush=True)
    model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained, device=device)
    model.eval()
    transform = preprocess

    def run_split(split_name, ids, img_root, out_path):
        dl = DataLoader(ImgDataset(ids, img_root, img_ext, transform), batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
        feats = np.zeros((len(ids), model.visual.output_dim), dtype=np.float32)
        t0 = time.time(); seen = 0
        with torch.no_grad():
            for bi, (xb, idx) in enumerate(dl):
                xb = xb.to(device, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=True):
                    f = model.encode_image(xb)
                f = torch.nn.functional.normalize(f.float(), dim=1)
                feats[idx.numpy()] = f.cpu().numpy()
                seen += xb.size(0)
                if (bi+1) % 20 == 0:
                    dt = time.time()-t0
                    print(f'[{split_name}] {seen}/{len(ids)} ({seen/max(dt,1e-6):.1f} img/s) elapsed {dt/60:.1f}m', flush=True)
        np.save(out_path, feats)
        print(f'[{split_name}] Saved {out_path} shape={feats.shape}', flush=True)
        del feats; gc.collect()

    train_ids = train_df['id'].tolist()
    test_ids = test_df['id'].tolist()
    if not Path('clip_train_emb.npy').exists():
        run_split('train', train_ids, Path('train'), 'clip_train_emb.npy')
    else:
        print('[train] clip_train_emb.npy exists, skipping')
    if not Path('clip_test_emb.npy').exists():
        run_split('test', test_ids, Path('test'), 'clip_test_emb.npy')
    else:
        print('[test] clip_test_emb.npy exists, skipping')
    print('CLIP embedding extraction done.')

print('Safe installer ready. Next: run extract_clip_embeddings() to generate CLIP embeddings, then build FAISS kNN and blend.', flush=True)

=== Safe install for CLIP kNN (no torch upgrade) and define extract ===


> pip install --no-deps open_clip_torch==2.26.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 55.6 MB/s eta 0:00:00
> pip install -c constraints.txt ftfy==6.3.1 tqdm huggingface-hub==0.35.1 regex safetensors pillow --upgrade-strategy only-if-needed


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 KB 82.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 KB 450.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 KB 294.3 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 KB 414.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.6/806.6 KB 505.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 KB 403.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 KB 488.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 401.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 465.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 404.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 441.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 469.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 179.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 260.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 396.0 MB/s eta 0:00:00


Safe installer ready. Next: run extract_clip_embeddings() to generate CLIP embeddings, then build FAISS kNN and blend.


In [58]:
print('=== Extracting CLIP embeddings (ViT-B/32 @224, BS=256) for kNN label transfer ===', flush=True)
try:
    extract_clip_embeddings(model_name='ViT-B-32', pretrained='laion2b_s34b_b79k', batch_size=256, num_workers=8, device='cuda')
except Exception as e:
    print('CLIP extraction failed:', e)

=== Extracting CLIP embeddings (ViT-B/32 @224, BS=256) for kNN label transfer ===


Detected ext: .png


Loading CLIP: ViT-B-32 laion2b_s34b_b79k


/app/.pip-target/open_clip/factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=map_location)


/tmp/ipykernel_107/1345938480.py:67: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


...

[test] 10240/21318 (1423.9 img/s) elapsed 0.1m


[test] 15360/21318 (1487.9 img/s) elapsed 0.2m


[test] 20480/21318 (1581.3 img/s) elapsed 0.2m


[test] Saved clip_test_emb.npy shape=(21318, 512)


CLIP embedding extraction done.


In [59]:
import numpy as np, pandas as pd, os, time, faiss, shutil
from pathlib import Path

print('=== kNN label transfer with CLIP + b3-only logit blend + v2 caps + tuned threshold ===', flush=True)

# 0) Preconditions: embeddings must exist
train_emb_p = Path('clip_train_emb.npy')
test_emb_p = Path('clip_test_emb.npy')
if not (train_emb_p.exists() and test_emb_p.exists()):
    print('[SKIP] Missing CLIP embeddings. Run extract_clip_embeddings() first.')
else:
    t0 = time.time()
    Xtr = np.load(train_emb_p).astype(np.float32)
    Xte = np.load(test_emb_p).astype(np.float32)
    # Assumed already L2-normalized in extractor
    print('Loaded embeddings:', Xtr.shape, Xte.shape, flush=True)

    # 1) Build FAISS index (cosine via inner-product on normalized vectors)
    d = Xtr.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(Xtr)
    print('FAISS index built. nt=', index.ntotal, 'dim=', d, flush=True)

    # 2) kNN search
    k = 20
    sims, nn_idx = index.search(Xte, k)  # sims in [-1,1] due to cosine
    sims = np.maximum(sims, 0.0).astype(np.float32)  # clamp negatives to 0
    print('kNN done. sims shape:', sims.shape, 'idx shape:', nn_idx.shape, flush=True)

    # 3) Build train label matrix (CSR-like via manual aggregation)
    labels_df = pd.read_csv('labels.csv')
    attr_ids_sorted = np.array(sorted(labels_df['attribute_id'].astype(int).unique().tolist()), dtype=np.int32)
    attr_to_col = {a:i for i,a in enumerate(attr_ids_sorted)}
    C = len(attr_ids_sorted)
    train_df = pd.read_csv('train.csv')
    n = len(train_df)
    # Prepare list of sets for memory efficiency during gather
    lab_sets = []
    for s in train_df['attribute_ids'].fillna('').astype(str).tolist():
        if s:
            lab_sets.append([attr_to_col[int(x)] for x in s.split() if x!='' and int(x) in attr_to_col])
        else:
            lab_sets.append([])
    print('Built train label indices (lists).', flush=True)

    # 4) Compute kNN label probabilities for test: weighted average over neighbors
    ntest = Xte.shape[0]
    probs_knn = np.zeros((ntest, C), dtype=np.float32)
    wsum = sims.sum(axis=1, keepdims=True) + 1e-8
    for i in range(ntest):
        nn = nn_idx[i]
        ws = sims[i] / wsum[i]
        # accumulate weighted labels
        for nbr, w in zip(nn, ws):
            for c in lab_sets[nbr]:
                probs_knn[i, c] += float(w)
        if (i+1) % 2000 == 0:
            print(f'[kNN] processed {i+1}/{ntest}', flush=True)
    print('kNN label transfer complete.', flush=True)

    # 5) Get b3-only model probs on test via our blender
    def sigmoid(x):
        return 1.0 / (1.0 + np.exp(-x))
    def probs_to_logits(p, eps=1e-5):
        p = np.clip(p, eps, 1.0 - eps)
        return np.log(p / (1.0 - p))
    from __main__ import blend_equal_weight  # defined earlier in this notebook
    dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
    _, _, Pt_b3 = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.40)
    print('Blender produced model probs:', None if Pt_b3 is None else Pt_b3.shape, flush=True)

    # 6) Logit blend: z_final = z_model + alpha * z_knn
    alpha = 0.5
    Zm = probs_to_logits(Pt_b3)
    Zk = probs_to_logits(probs_knn)
    Zf = Zm + alpha * Zk
    Pf = sigmoid(Zf)
    del Zm, Zk, Zf;

    # 7) v2 caps + post-cap threshold sweep to target mean ~4.40
    labels_df = pd.read_csv('labels.csv')
    labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
    idx_to_group = labels_df.set_index('attribute_id').loc[attr_ids_sorted, 'group'].values
    caps = {
        'country': 1,
        'culture': 1,
        'medium': 2,
        'dimension': 1,
        'tags': 5,
        'tag': 5,
    }
    default_cap = 3
    sub = pd.read_csv('sample_submission.csv')
    ids = sub['id'].values

    def apply_caps(Pt, thr):
        rows = []; counts = []
        for i in range(Pt.shape[0]):
            p = Pt[i]
            cand = np.where(p >= thr)[0]
            if cand.size == 0:
                cand = np.array([int(np.argmax(p))], dtype=np.int64)
            cand_sorted = cand[np.argsort(-p[cand])]
            used, kept = {}, []
            for j in cand_sorted:
                g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
                kcap = caps.get(g, default_cap)
                c = used.get(g, 0)
                if c < kcap:
                    kept.append(j); used[g] = c + 1
            if len(kept) == 0:
                kept = [int(cand_sorted[0])]
            pred_attr = [int(attr_ids_sorted[j]) for j in kept]
            rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(x) for x in pred_attr)})
            counts.append(len(pred_attr))
        return rows, float(np.mean(counts))

    thrs = np.arange(0.460, 0.5201, 0.002)
    target_mean = 4.40
    best = None; best_rows = None
    for t in thrs:
        rows_t, mean_t = apply_caps(Pf, t)
        delta = abs(mean_t - target_mean)
        if (best is None) or (delta < best[0]):
            best = (delta, t, mean_t); best_rows = rows_t
        # periodic log
        if int((t-0.460)/0.002) % 10 == 0:
            print(f'[SWEEP] thr={t:.3f} post-cap mean={mean_t:.3f}')
    _, best_thr_cap, best_mean = best
    print(f'[CHOSEN] thr={best_thr_cap:.3f} post-cap mean={best_mean:.3f}')
    sub_df = pd.DataFrame(best_rows)
    out_path = 'submission_knn_blend_tuned.csv'
    sub_df.to_csv(out_path, index=False)
    shutil.copyfile(out_path, 'submission.csv')
    print('Wrote', out_path, 'and set as submission.csv. Elapsed:', round((time.time()-t0)/60,1),'min', flush=True)

=== kNN label transfer with CLIP + b3-only logit blend + v2 caps + tuned threshold ===


Loaded embeddings: (120801, 512) (21318, 512)


FAISS index built. nt= 120801 dim= 512


kNN done. sims shape: (21318, 20) idx shape: (21318, 20)


Built train label indices (lists).


...

[CARD] Train mean labels/img=4.421, target=4.400, chosen thr=0.502 (pred_mean=4.406)
[INFO] Skipping submission write (best_thr not available).
Blender produced model probs: (21318, 3474)


[SWEEP] thr=0.460 post-cap mean=3.097


[SWEEP] thr=0.480 post-cap mean=2.995


[SWEEP] thr=0.502 post-cap mean=2.888


[SWEEP] thr=0.520 post-cap mean=2.804
[CHOSEN] thr=0.460 post-cap mean=3.097
Wrote submission_knn_blend_tuned.csv and set as submission.csv. Elapsed: 1.2 min


In [61]:
import numpy as np, pandas as pd, shutil
from pathlib import Path

print('=== B3-only v2 caps @ target=4.38, weights 1:1, wide post-cap thr sweep to hit mean~4.40 ===', flush=True)
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
oof_f1_b3, thr_b3_base, Pt_b3 = blend_equal_weight(dirs_b3, weights=[1,1], write_submission=False, out_name='noop.csv', cardinality_target=4.38)
print('[BLEND B3 1:1] base thr=', thr_b3_base, 'OOF_f1=', oof_f1_b3)

labels_df = pd.read_csv('labels.csv')
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
attr_ids = labels_df['attribute_id'].astype(int).tolist()
idx_to_attr = np.array(sorted(attr_ids), dtype=np.int32)
idx_to_group = labels_df.set_index('attribute_id').loc[idx_to_attr, 'group'].values
caps = {
    'country': 1,
    'culture': 1,
    'medium': 2,
    'dimension': 1,
    'tags': 5,
    'tag': 5,
}
default_cap = 3

sub = pd.read_csv('sample_submission.csv')
ids = sub['id'].values

def apply_caps(Pt, thr):
    rows = []; counts = []
    for i in range(len(ids)):
        p = Pt[i]
        cand = np.where(p >= thr)[0]
        if cand.size == 0:
            cand = np.array([int(np.argmax(p))], dtype=np.int64)
        cand_sorted = cand[np.argsort(-p[cand])]
        used, kept = {}, []
        for j in cand_sorted:
            g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
            kcap = caps.get(g, default_cap)
            c = used.get(g, 0)
            if c < kcap:
                kept.append(j); used[g] = c + 1
        if len(kept) == 0:
            kept = [int(cand_sorted[0])]
        pred_attr = [int(idx_to_attr[j]) for j in kept]
        rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(x) for x in pred_attr)})
        counts.append(len(pred_attr))
    return rows, float(np.mean(counts))

# Wide sweep to reach post-cap mean ~4.40
thrs = np.arange(0.460, 0.5201, 0.002)
target_mean = 4.40
best = None; best_rows = None
for t in thrs:
    rows_t, mean_t = apply_caps(Pt_b3, t)
    delta = abs(mean_t - target_mean)
    if (best is None) or (delta < best[0]):
        best = (delta, t, mean_t); best_rows = rows_t
    if int((t-0.460)/0.002) % 10 == 0:
        print(f'[SWEEP] thr={t:.3f} post-cap mean={mean_t:.3f}')

_, best_thr_cap, best_mean = best
print(f'[CHOSEN] thr={best_thr_cap:.3f} post-cap mean={best_mean:.3f}')
sub_df = pd.DataFrame(best_rows)
out_path = 'submission_b3_caps_438_w11_tuned.csv'
sub_df.to_csv(out_path, index=False)
shutil.copyfile(out_path, 'submission.csv')
print('Wrote', out_path, 'and set as submission.csv')

=== B3-only v2 caps @ target=4.38, weights 1:1, wide post-cap thr sweep to hit mean~4.40 ===


[CARD] Train mean labels/img=4.421, target=4.380, chosen thr=0.504 (pred_mean=4.394)
[INFO] Skipping submission write (best_thr not available).
[BLEND B3 1:1] base thr= 0.504 OOF_f1= None


[SWEEP] thr=0.460 post-cap mean=4.456


[SWEEP] thr=0.480 post-cap mean=4.197


[SWEEP] thr=0.502 post-cap mean=3.922


[SWEEP] thr=0.520 post-cap mean=3.711
[CHOSEN] thr=0.464 post-cap mean=4.400
Wrote submission_b3_caps_438_w11_tuned.csv and set as submission.csv


In [62]:
import numpy as np, pandas as pd, os, time, faiss, shutil
from pathlib import Path

print('=== kNN label transfer v2: k=50, alpha=0.3, b3-only blend, v2 caps, tuned threshold ===', flush=True)

train_emb_p = Path('clip_train_emb.npy')
test_emb_p = Path('clip_test_emb.npy')
if not (train_emb_p.exists() and test_emb_p.exists()):
    print('[SKIP] Missing CLIP embeddings.')
else:
    t0 = time.time()
    Xtr = np.load(train_emb_p).astype(np.float32)
    Xte = np.load(test_emb_p).astype(np.float32)
    print('Loaded embeddings:', Xtr.shape, Xte.shape, flush=True)

    d = Xtr.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(Xtr)
    print('FAISS index built. nt=', index.ntotal, 'dim=', d, flush=True)

    k = 50
    sims, nn_idx = index.search(Xte, k)
    sims = np.maximum(sims, 0.0).astype(np.float32)
    print('kNN done. sims shape:', sims.shape, 'idx shape:', nn_idx.shape, flush=True)

    labels_df = pd.read_csv('labels.csv')
    attr_ids_sorted = np.array(sorted(labels_df['attribute_id'].astype(int).unique().tolist()), dtype=np.int32)
    attr_to_col = {a:i for i,a in enumerate(attr_ids_sorted)}
    C = len(attr_ids_sorted)
    train_df = pd.read_csv('train.csv')
    lab_lists = []
    for s in train_df['attribute_ids'].fillna('').astype(str).tolist():
        if s:
            lab_lists.append([attr_to_col[int(x)] for x in s.split() if x!='' and int(x) in attr_to_col])
        else:
            lab_lists.append([])
    print('Built train label lists.', flush=True)

    ntest = Xte.shape[0]
    probs_knn = np.zeros((ntest, C), dtype=np.float32)
    wsum = sims.sum(axis=1, keepdims=True) + 1e-8
    for i in range(ntest):
        nn = nn_idx[i]
        ws = sims[i] / wsum[i]
        for nbr, w in zip(nn, ws):
            for c in lab_lists[nbr]:
                probs_knn[i, c] += float(w)
        if (i+1) % 2000 == 0:
            print(f'[kNN] {i+1}/{ntest}', flush=True)

    def sigmoid(x):
        return 1.0 / (1.0 + np.exp(-x))
    def probs_to_logits(p, eps=1e-5):
        p = np.clip(p, eps, 1.0 - eps)
        return np.log(p / (1.0 - p))

    from __main__ import blend_equal_weight
    dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
    _, _, Pt_b3 = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.40)
    print('Model probs shape:', None if Pt_b3 is None else Pt_b3.shape, flush=True)

    alpha = 0.3
    Zm = probs_to_logits(Pt_b3)
    Zk = probs_to_logits(probs_knn)
    Pf = sigmoid(Zm + alpha * Zk)
    del Zm, Zk

    labels_df = pd.read_csv('labels.csv')
    labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
    idx_to_group = labels_df.set_index('attribute_id').loc[attr_ids_sorted, 'group'].values
    caps = {
        'country': 1,
        'culture': 1,
        'medium': 2,
        'dimension': 1,
        'tags': 5,
        'tag': 5,
    }
    default_cap = 3
    sub = pd.read_csv('sample_submission.csv')
    ids = sub['id'].values

    def apply_caps(Pt, thr):
        rows = []; counts = []
        for i in range(Pt.shape[0]):
            p = Pt[i]
            cand = np.where(p >= thr)[0]
            if cand.size == 0:
                cand = np.array([int(np.argmax(p))], dtype=np.int64)
            cand_sorted = cand[np.argsort(-p[cand])]
            used, kept = {}, []
            for j in cand_sorted:
                g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
                kcap = caps.get(g, default_cap)
                c = used.get(g, 0)
                if c < kcap:
                    kept.append(j); used[g] = c + 1
            if len(kept) == 0:
                kept = [int(cand_sorted[0])]
            pred_attr = [int(attr_ids_sorted[j]) for j in kept]
            rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(x) for x in pred_attr)})
            counts.append(len(pred_attr))
        return rows, float(np.mean(counts))

    thrs = np.arange(0.440, 0.5401, 0.002)
    target_mean = 4.40
    best = None; best_rows = None
    for t in thrs:
        rows_t, mean_t = apply_caps(Pf, t)
        delta = abs(mean_t - target_mean)
        if (best is None) or (delta < best[0]):
            best = (delta, t, mean_t); best_rows = rows_t
        if int((t-0.440)/0.002) % 15 == 0:
            print(f'[SWEEP] thr={t:.3f} post-cap mean={mean_t:.3f}')

    _, best_thr_cap, best_mean = best
    print(f'[CHOSEN] thr={best_thr_cap:.3f} post-cap mean={best_mean:.3f}')
    sub_df = pd.DataFrame(best_rows)
    out_path = 'submission_knn_blend_tuned_v2.csv'
    sub_df.to_csv(out_path, index=False)
    shutil.copyfile(out_path, 'submission.csv')
    print('Wrote', out_path, 'and set as submission.csv. Elapsed:', round((time.time()-t0)/60,1), 'min', flush=True)

=== kNN label transfer v2: k=50, alpha=0.3, b3-only blend, v2 caps, tuned threshold ===


Loaded embeddings: (120801, 512) (21318, 512)


FAISS index built. nt= 120801 dim= 512


kNN done. sims shape: (21318, 50) idx shape: (21318, 50)


Built train label lists.


...

[SWEEP] thr=0.440 post-cap mean=3.431


[SWEEP] thr=0.470 post-cap mean=3.231


[SWEEP] thr=0.500 post-cap mean=3.040


[SWEEP] thr=0.530 post-cap mean=2.860


[CHOSEN] thr=0.440 post-cap mean=3.431
Wrote submission_knn_blend_tuned_v2.csv and set as submission.csv. Elapsed: 1.4 min


In [63]:
import numpy as np, pandas as pd, os, time, faiss, shutil
from pathlib import Path

print('=== kNN label transfer v3: softmax-weighted (tau=0.10) + IDF^0.5 + per-neighbor |L|^-1 + prob-blend beta=0.10 + caps + min-total ===', flush=True)

t0 = time.time()
train_emb_p = Path('clip_train_emb.npy')
test_emb_p = Path('clip_test_emb.npy')
assert train_emb_p.exists() and test_emb_p.exists(), 'Missing CLIP embeddings; run extraction first.'
Xtr = np.load(train_emb_p).astype(np.float32)
Xte = np.load(test_emb_p).astype(np.float32)
print('Loaded embeddings:', Xtr.shape, Xte.shape, flush=True)

# Build FAISS IP (cosine on L2-normalized feats)
d = Xtr.shape[1]
index = faiss.IndexFlatIP(d)
index.add(Xtr)
print('FAISS index built. nt=', index.ntotal, 'dim=', d, flush=True)

# Parameters per expert
k = 100
tau = 0.10
eta = 1.0  # per-neighbor normalization by |L_j|^eta
gamma_idf = 0.5

# kNN search
sims, nn_idx = index.search(Xte, k)  # cosine similarities
print('kNN done. sims shape:', sims.shape, 'idx shape:', nn_idx.shape, flush=True)

# Labels and priors
labels_df = pd.read_csv('labels.csv')
attr_ids_sorted = np.array(sorted(labels_df['attribute_id'].astype(int).unique().tolist()), dtype=np.int32)
attr_to_col = {a:i for i,a in enumerate(attr_ids_sorted)}
C = len(attr_ids_sorted)
train_df = pd.read_csv('train.csv')
n_train = len(train_df)

# Build train label lists and |L_j|
lab_lists = []
lab_sizes = np.zeros(n_train, dtype=np.int32)
df_counts = np.zeros(C, dtype=np.int32)  # document frequency per class
for i, s in enumerate(train_df['attribute_ids'].fillna('').astype(str).tolist()):
    if s:
        cols = [attr_to_col[int(x)] for x in s.split() if x!='' and int(x) in attr_to_col]
        lab_lists.append(cols)
        lab_sizes[i] = len(cols)
        for c in set(cols):
            df_counts[c] += 1
    else:
        lab_lists.append([])
        lab_sizes[i] = 0

# IDF weights
idf = np.log(n_train / (df_counts.astype(np.float32) + 1.0))
idf = np.clip(idf, 0.0, None) ** gamma_idf  # idf^gamma

# Softmax weights over neighbors with temperature tau
sims_sm = sims / max(tau, 1e-6)
sims_sm = sims_sm - sims_sm.max(axis=1, keepdims=True)  # stabilize
w = np.exp(sims_sm)
w /= (w.sum(axis=1, keepdims=True) + 1e-12)

# Optional prune by cumulative weight >= 0.95 (skip for speed/consistency)

# Accumulate class scores with per-neighbor normalization by |L_j|^eta
ntest = Xte.shape[0]
probs_knn = np.zeros((ntest, C), dtype=np.float32)
for i in range(ntest):
    nn = nn_idx[i]
    wi = w[i]
    for rank, (nbr, wj) in enumerate(zip(nn, wi)):
        Lj = lab_lists[nbr]
        if not Lj:
            continue
        denom = (lab_sizes[nbr] ** eta) if eta > 0 else 1.0
        add = float(wj) / float(max(denom, 1.0))
        for c in Lj:
            probs_knn[i, c] += add
    if (i+1) % 2000 == 0:
        print(f'[kNN-accum] {i+1}/{ntest}', flush=True)

# Apply IDF and renormalize per image to sum 1
probs_knn *= idf[None, :]
row_sums = probs_knn.sum(axis=1, keepdims=True)
probs_knn = np.divide(probs_knn, np.where(row_sums > 0, row_sums, 1.0), out=np.zeros_like(probs_knn), where=row_sums>0)

# Load b3-only model probs and do probability-space blend
from __main__ import blend_equal_weight
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
_, _, P_model = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.40)
assert P_model is not None and P_model.shape == probs_knn.shape, f'Shape mismatch: {None if P_model is None else P_model.shape} vs {probs_knn.shape}'
beta = 0.10
P_final = (1.0 - beta) * P_model + beta * probs_knn

# Caps + min-total enforcement
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
idx_to_group = labels_df.set_index('attribute_id').loc[attr_ids_sorted, 'group'].values
base_caps = {
    'country': 1,
    'culture': 1,
    'medium': 2,
    'dimension': 1,
    'tags': 5,
    'tag': 5,
}
default_cap = 3
sub = pd.read_csv('sample_submission.csv')
ids = sub['id'].values

def apply_caps_with_min(Pt, thr, caps, min_total, fill_probs):
    rows = []; counts = []
    for i in range(Pt.shape[0]):
        p = Pt[i]
        cand = np.where(p >= thr)[0]
        if cand.size == 0:
            cand = np.array([int(np.argmax(p))], dtype=np.int64)
        cand_sorted = cand[np.argsort(-p[cand])]
        used, kept = {}, []
        kept_set = set()
        for j in cand_sorted:
            g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
            kcap = caps.get(g, default_cap)
            c = used.get(g, 0)
            if c < kcap:
                kept.append(j); kept_set.add(j); used[g] = c + 1
        # Enforce min_total by filling from highest model probs not yet selected
        if len(kept) < min_total:
            order = np.argsort(-fill_probs[i])
            for j in order:
                if j in kept_set:
                    continue
                g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
                kcap = caps.get(g, default_cap)
                c = used.get(g, 0)
                if c < kcap:
                    kept.append(j); kept_set.add(j); used[g] = c + 1
                    if len(kept) >= min_total:
                        break
        if len(kept) == 0:
            kept = [int(np.argmax(p))]
        pred_attr = [int(attr_ids_sorted[j]) for j in kept]
        rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(x) for x in pred_attr)})
        counts.append(len(pred_attr))
    return rows, float(np.mean(counts))

# Sweep threshold to hit ~4.40 with min_total=3; if too low, relax tags cap and min_total=4
def sweep_and_write(Pt, caps, min_total, out_path, thr_lo=0.46, thr_hi=0.54, thr_step=0.002):
    thrs = np.arange(thr_lo, thr_hi + 1e-9, thr_step)
    target_mean = 4.40
    best = None; best_rows = None
    for t in thrs:
        rows_t, mean_t = apply_caps_with_min(Pt, t, caps, min_total, fill_probs=P_model)
        delta = abs(mean_t - target_mean)
        if (best is None) or (delta < best[0]):
            best = (delta, t, mean_t); best_rows = rows_t
    _, best_thr, best_mean = best
    sub_df = pd.DataFrame(best_rows)
    sub_df.to_csv(out_path, index=False)
    shutil.copyfile(out_path, 'submission.csv')
    print(f'[WRITE] {out_path} thr={best_thr:.3f} post-cap mean={best_mean:.3f} min_total={min_total} caps[tags]={caps.get("tags", None)}')
    return best_thr, best_mean

# Attempt 1: base caps, min_total=3
thr1, mean1 = sweep_and_write(P_final, base_caps.copy(), min_total=3, out_path='submission_knn_softmax_probblend.csv')

# If still under ~4.35, try tags cap=6 and/or min_total=4
if mean1 < 4.35:
    caps2 = base_caps.copy(); caps2['tags'] = 6; caps2['tag'] = 6
    thr2, mean2 = sweep_and_write(P_final, caps2, min_total=3, out_path='submission_knn_softmax_probblend_tags6.csv')
    if mean2 < 4.35:
        thr3, mean3 = sweep_and_write(P_final, caps2, min_total=4, out_path='submission_knn_softmax_probblend_tags6_min4.csv')

print('Done. Total elapsed: {:.1f} min'.format((time.time()-t0)/60.0), flush=True)

=== kNN label transfer v3: softmax-weighted (tau=0.10) + IDF^0.5 + per-neighbor |L|^-1 + prob-blend beta=0.10 + caps + min-total ===


Loaded embeddings: (120801, 512) (21318, 512)


FAISS index built. nt= 120801 dim= 512


kNN done. sims shape: (21318, 100) idx shape: (21318, 100)


[kNN-accum] 2000/21318


...

[kNN-accum] 20000/21318


[CARD] Train mean labels/img=4.421, target=4.400, chosen thr=0.502 (pred_mean=4.406)
[INFO] Skipping submission write (best_thr not available).


[WRITE] submission_knn_softmax_probblend.csv thr=0.460 post-cap mean=4.087 min_total=3 caps[tags]=5


[WRITE] submission_knn_softmax_probblend_tags6.csv thr=0.460 post-cap mean=4.093 min_total=3 caps[tags]=6


[WRITE] submission_knn_softmax_probblend_tags6_min4.csv thr=0.482 post-cap mean=4.395 min_total=4 caps[tags]=6
Done. Total elapsed: 5.0 min


In [ ]:
import numpy as np, pandas as pd, shutil
from pathlib import Path

print('=== Post-process: enforce min-total=4 on kNN softmax blend (tags cap=6) using P_model for fills ===', flush=True)

# Inputs
sub_path = Path('submission_knn_softmax_probblend_tags6.csv')
assert sub_path.exists(), 'Missing submission_knn_softmax_probblend_tags6.csv from kNN v3 cell'

# Load mapping and model probabilities for fill ranking
labels_df = pd.read_csv('labels.csv')
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
attr_ids_sorted = np.array(sorted(labels_df['attribute_id'].astype(int).unique().tolist()), dtype=np.int32)
attr_to_col = {a:i for i,a in enumerate(attr_ids_sorted)}
idx_to_group = labels_df.set_index('attribute_id').loc[attr_ids_sorted, 'group'].values
default_cap = 3
caps = {'country':1,'culture':1,'medium':2,'dimension':1,'tags':6,'tag':6}

# Get model-only probs for ranking fills
from __main__ import blend_equal_weight
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
_, _, P_model = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.40)
assert P_model is not None, 'Failed to load model probabilities'

# Load existing submission
sub = pd.read_csv(sub_path)
ids = sub['id'].values
pred_lists = [str(x).strip().split() if isinstance(x, str) else [] for x in sub['attribute_ids'].values]

rows = []; counts = []
for i, (img_id, attrs) in enumerate(zip(ids, pred_lists)):
    chosen_attr_ids = [int(a) for a in attrs if a!='']
    chosen_idx = [attr_to_col[a] for a in chosen_attr_ids if a in attr_to_col]
    used = {}
    for j in chosen_idx:
        g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
        used[g] = used.get(g, 0) + 1
    # Fill until min-total=4 respecting caps
    while len(chosen_idx) < 4:
        # pick next best by P_model
        order = np.argsort(-P_model[i])
        picked = False
        for j in order:
            if j in chosen_idx:
                continue
            g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
            kcap = caps.get(g, default_cap)
            c = used.get(g, 0)
            if c < kcap:
                chosen_idx.append(int(j)); used[g] = c + 1; picked = True
                break
        if not picked:
            break  # no more slots available due to caps
    pred_attr = [int(attr_ids_sorted[j]) for j in chosen_idx]
    rows.append({'id': img_id, 'attribute_ids': ' '.join(str(x) for x in pred_attr)})
    counts.append(len(pred_attr))

out_path = 'submission_knn_softmax_probblend_tags6_min4_fill.csv'
pd.DataFrame(rows).to_csv(out_path, index=False)
shutil.copyfile(out_path, 'submission.csv')
print(f'Wrote {out_path} (mean count={np.mean(counts):.3f}) and set as submission.csv')

In [64]:
import numpy as np, pandas as pd, time, shutil, faiss
from pathlib import Path

print('=== kNN union hedge: add up to L=2 kNN labels (P_knn>=0.10) to model+caps; sweep thr to mean~4.40 ===', flush=True)
t0 = time.time()

# Load CLIP embeddings
Xtr = np.load('clip_train_emb.npy').astype(np.float32)
Xte = np.load('clip_test_emb.npy').astype(np.float32)
d = Xtr.shape[1]
index = faiss.IndexFlatIP(d); index.add(Xtr)

# Params (per expert): k=150, tau=0.10, eta=1.0, idf^0.5
k = 150; tau = 0.10; eta = 1.0; gamma_idf = 0.5
sims, nn_idx = index.search(Xte, k)
print('kNN search done:', sims.shape, flush=True)

# Labels mapping
labels_df = pd.read_csv('labels.csv')
attr_ids_sorted = np.array(sorted(labels_df['attribute_id'].astype(int).unique().tolist()), dtype=np.int32)
attr_to_col = {a:i for i,a in enumerate(attr_ids_sorted)}
C = len(attr_ids_sorted)
train_df = pd.read_csv('train.csv')
n_train = len(train_df)

# Train label lists and sizes + df counts
lab_lists = []; lab_sizes = np.zeros(n_train, dtype=np.int32); df_counts = np.zeros(C, dtype=np.int32)
for i, s in enumerate(train_df['attribute_ids'].fillna('').astype(str).tolist()):
    if s:
        cols = [attr_to_col[int(x)] for x in s.split() if x!='' and int(x) in attr_to_col]
        lab_lists.append(cols); lab_sizes[i] = len(cols)
        for c in set(cols): df_counts[c] += 1
    else:
        lab_lists.append([]); lab_sizes[i] = 0

# Softmax weights with temperature
sims_sm = sims / max(tau,1e-6)
sims_sm -= sims_sm.max(axis=1, keepdims=True)
w = np.exp(sims_sm); w /= (w.sum(axis=1, keepdims=True) + 1e-12)

# Accumulate probs_knn with per-neighbor |L|^-eta
ntest = Xte.shape[0]
probs_knn = np.zeros((ntest, C), dtype=np.float32)
for i in range(ntest):
    nn = nn_idx[i]; wi = w[i]
    for nbr, wj in zip(nn, wi):
        Lj = lab_lists[nbr]
        if not Lj: continue
        denom = (lab_sizes[nbr] ** eta) if eta > 0 else 1.0
        add = float(wj) / float(max(denom, 1.0))
        for c in Lj: probs_knn[i, c] += add
    if (i+1) % 2000 == 0: print(f'[kNN-accum] {i+1}/{ntest}', flush=True)

# IDF^gamma and renormalize
idf = np.log(n_train / (df_counts.astype(np.float32) + 1.0))
idf = np.clip(idf, 0.0, None) ** gamma_idf
probs_knn *= idf[None, :]
row_sums = probs_knn.sum(axis=1, keepdims=True)
probs_knn = np.divide(probs_knn, np.where(row_sums > 0, row_sums, 1.0), out=np.zeros_like(probs_knn), where=row_sums>0)

# Load model probabilities (b3 2:1)
from __main__ import blend_equal_weight
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
_, _, P_model = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.40)
assert P_model is not None and P_model.shape == probs_knn.shape

# Groups and caps (tags cap=6 as per expert bump), default cap=3
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
idx_to_group = labels_df.set_index('attribute_id').loc[attr_ids_sorted, 'group'].values
caps = {'country':1,'culture':1,'medium':2,'dimension':1,'tags':6,'tag':6}
default_cap = 3
ids = pd.read_csv('sample_submission.csv')['id'].values

def build_union_rows(thr, L=2, gate=0.10, min_total=3):
    rows = []; counts = []
    for i in range(P_model.shape[0]):
        p = P_model[i]; pk = probs_knn[i]
        cand = np.where(p >= thr)[0]
        if cand.size == 0: cand = np.array([int(np.argmax(p))], dtype=np.int64)
        cand_sorted = cand[np.argsort(-p[cand])]
        used = {}; kept = []; kept_set = set()
        # apply caps on model-selected first
        for j in cand_sorted:
            g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
            kcap = caps.get(g, default_cap); c = used.get(g, 0)
            if c < kcap: kept.append(j); kept_set.add(j); used[g] = c + 1
        # union: add up to L labels from kNN by pk desc with gate
        if L > 0:
            order_knn = np.argsort(-pk)
            added = 0
            for j in order_knn:
                if pk[j] < gate: break
                if j in kept_set: continue
                g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
                kcap = caps.get(g, default_cap); c = used.get(g, 0)
                if c < kcap:
                    kept.append(int(j)); kept_set.add(int(j)); used[g] = c + 1; added += 1
                    if added >= L: break
        # enforce min_total using model ranking
        if len(kept) < min_total:
            order = np.argsort(-p)
            for j in order:
                if j in kept_set: continue
                g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
                kcap = caps.get(g, default_cap); c = used.get(g, 0)
                if c < kcap:
                    kept.append(int(j)); kept_set.add(int(j)); used[g] = c + 1
                    if len(kept) >= min_total: break
        if len(kept) == 0: kept = [int(np.argmax(p))]
        rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(int(attr_ids_sorted[j])) for j in kept)})
        counts.append(len(kept))
    return rows, float(np.mean(counts))

# Sweep thr to target mean ~4.40 after union; L=2, gate=0.10, min_total=3
thrs = np.arange(0.460, 0.5401, 0.002)
target_mean = 4.40
best = None; best_rows = None
for t in thrs:
    rows_t, mean_t = build_union_rows(t, L=2, gate=0.10, min_total=3)
    delta = abs(mean_t - target_mean)
    if (best is None) or (delta < best[0]):
        best = (delta, t, mean_t); best_rows = rows_t
    if int((t-0.460)/0.002) % 10 == 0:
        print(f'[SWEEP] thr={t:.3f} post-union mean={mean_t:.3f}', flush=True)

_, best_thr, best_mean = best
out_path = 'submission_union_knn_add2.csv'
pd.DataFrame(best_rows).to_csv(out_path, index=False)
shutil.copyfile(out_path, 'submission.csv')
print(f'[WRITE] {out_path} thr={best_thr:.3f} post-union mean={best_mean:.3f} | Elapsed {(time.time()-t0)/60:.1f}m', flush=True)

=== kNN union hedge: add up to L=2 kNN labels (P_knn>=0.10) to model+caps; sweep thr to mean~4.40 ===


kNN search done: (21318, 150)


[kNN-accum] 2000/21318


[kNN-accum] 4000/21318


[kNN-accum] 6000/21318


...

[SWEEP] thr=0.480 post-union mean=4.458


[SWEEP] thr=0.502 post-union mean=4.234


[SWEEP] thr=0.520 post-union mean=4.070


[SWEEP] thr=0.540 post-union mean=3.919


[WRITE] submission_union_knn_add2.csv thr=0.486 post-union mean=4.395 | Elapsed 2.7m


In [65]:
import numpy as np, pandas as pd, time, shutil, faiss
from pathlib import Path

print('=== kNN v3 variants: (A) tau=0.05, beta=0.10; (B) tau=0.10, beta=0.15, restrict to {country,culture,dimension} ===', flush=True)
t0 = time.time()

# Common: load embeddings and build FAISS once with k_max=150
Xtr = np.load('clip_train_emb.npy').astype(np.float32)
Xte = np.load('clip_test_emb.npy').astype(np.float32)
d = Xtr.shape[1]
index = faiss.IndexFlatIP(d); index.add(Xtr)
k_max = 150
sims_all, nn_idx_all = index.search(Xte, k_max)
print('FAISS kNN ready:', sims_all.shape, flush=True)

# Labels and train annotations
labels_df = pd.read_csv('labels.csv')
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
attr_ids_sorted = np.array(sorted(labels_df['attribute_id'].astype(int).unique().tolist()), dtype=np.int32)
attr_to_col = {a:i for i,a in enumerate(attr_ids_sorted)}
idx_to_group = labels_df.set_index('attribute_id').loc[attr_ids_sorted, 'group'].values
C = len(attr_ids_sorted)
train_df = pd.read_csv('train.csv')
n_train = len(train_df)
lab_lists = []; lab_sizes = np.zeros(n_train, dtype=np.int32); df_counts = np.zeros(C, dtype=np.int32)
for i, s in enumerate(train_df['attribute_ids'].fillna('').astype(str).tolist()):
    if s:
        cols = [attr_to_col.get(int(x)) for x in s.split() if x!='' and (int(x) in attr_to_col)]
        cols = [c for c in cols if c is not None]
        lab_lists.append(cols); lab_sizes[i] = len(cols)
        for c in set(cols): df_counts[c] += 1
    else:
        lab_lists.append([]); lab_sizes[i] = 0
idf = np.log(n_train / (df_counts.astype(np.float32) + 1.0))
idf = np.clip(idf, 0.0, None) ** 0.5  # gamma_idf=0.5

# Load model probabilities (b3 2:1)
from __main__ import blend_equal_weight
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
_, _, P_model = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.40)
assert P_model is not None and P_model.shape[1] == C

sub = pd.read_csv('sample_submission.csv'); ids = sub['id'].values
base_caps = {'country':1,'culture':1,'medium':2,'dimension':1,'tags':6,'tag':6}
default_cap = 3

def apply_caps_with_min(Pt, thr, caps, min_total, fill_probs):
    rows = []; counts = []
    for i in range(Pt.shape[0]):
        p = Pt[i]
        cand = np.where(p >= thr)[0]
        if cand.size == 0:
            cand = np.array([int(np.argmax(p))], dtype=np.int64)
        cand_sorted = cand[np.argsort(-p[cand])]
        used, kept = {}, []; kept_set = set()
        for j in cand_sorted:
            g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
            kcap = caps.get(g, default_cap); c = used.get(g, 0)
            if c < kcap:
                kept.append(j); kept_set.add(j); used[g] = c + 1
        if len(kept) < min_total:
            order = np.argsort(-fill_probs[i])
            for j in order:
                if j in kept_set: continue
                g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
                kcap = caps.get(g, default_cap); c = used.get(g, 0)
                if c < kcap:
                    kept.append(j); kept_set.add(j); used[g] = c + 1
                    if len(kept) >= min_total: break
        if len(kept) == 0:
            kept = [int(np.argmax(p))]
        rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(int(attr_ids_sorted[j])) for j in kept)})
        counts.append(len(kept))
    return rows, float(np.mean(counts))

def sweep_and_write(Pt, caps, min_total, out_path, thr_lo=0.46, thr_hi=0.54, thr_step=0.002):
    thrs = np.arange(thr_lo, thr_hi + 1e-9, thr_step)
    target_mean = 4.40
    best = None; best_rows = None
    for t in thrs:
        rows_t, mean_t = apply_caps_with_min(Pt, t, caps, min_total, fill_probs=P_model)
        delta = abs(mean_t - target_mean)
        if (best is None) or (delta < best[0]):
            best = (delta, t, mean_t); best_rows = rows_t
    _, best_thr, best_mean = best
    pd.DataFrame(best_rows).to_csv(out_path, index=False)
    shutil.copyfile(out_path, 'submission.csv')
    print(f'[WRITE] {out_path} thr={best_thr:.3f} post-cap mean={best_mean:.3f} min_total={min_total}')
    return best_thr, best_mean

def build_probs_knn(k_use, tau):
    sims = sims_all[:, :k_use]
    nn_idx = nn_idx_all[:, :k_use]
    sims_sm = sims / max(tau, 1e-6)
    sims_sm -= sims_sm.max(axis=1, keepdims=True)
    w = np.exp(sims_sm); w /= (w.sum(axis=1, keepdims=True) + 1e-12)
    ntest = sims.shape[0]
    pk = np.zeros((ntest, C), dtype=np.float32)
    for i in range(ntest):
        wi = w[i]; nn = nn_idx[i]
        for nbr, wj in zip(nn, wi):
            Lj = lab_lists[nbr]
            if not Lj: continue
            denom = float(max(lab_sizes[nbr], 1))
            add = float(wj) / denom
            for c in Lj: pk[i, c] += add
        if (i+1) % 2000 == 0: print(f'[accum] {i+1}/{ntest}', flush=True)
    pk *= idf[None, :]
    rs = pk.sum(axis=1, keepdims=True)
    pk = np.divide(pk, np.where(rs>0, rs, 1.0), out=np.zeros_like(pk), where=rs>0)
    return pk

# Variant A: tau=0.05, k=150, beta=0.10, min_total=4, caps tags=6
print('--- Variant A: tau=0.05, k=150, beta=0.10 ---', flush=True)
Pk_A = build_probs_knn(150, 0.05)
beta_A = 0.10
P_final_A = (1.0 - beta_A) * P_model + beta_A * Pk_A
sweep_and_write(P_final_A, base_caps, min_total=4, out_path='submission_knn_softmax_probblend_tau005.csv')

# Variant B: tau=0.10, k=150, beta=0.15, restrict kNN to only country/culture/dimension
print('--- Variant B: tau=0.10, k=150, beta=0.15, restrict groups ---', flush=True)
Pk_B = build_probs_knn(150, 0.10)
mask_groups = np.isin(idx_to_group, ['country','culture','dimension'])
Pk_B_masked = Pk_B.copy(); Pk_B_masked[:, ~mask_groups] = 0.0
beta_B = 0.15
P_final_B = (1.0 - beta_B) * P_model + beta_B * Pk_B_masked
sweep_and_write(P_final_B, base_caps, min_total=3, out_path='submission_knn_softmax_probblend_groupmask.csv')

print('Done variants. Elapsed {:.1f}m'.format((time.time()-t0)/60.0), flush=True)

=== kNN v3 variants: (A) tau=0.05, beta=0.10; (B) tau=0.10, beta=0.15, restrict to {country,culture,dimension} ===


FAISS kNN ready: (21318, 150)


[CARD] Train mean labels/img=4.421, target=4.400, chosen thr=0.502 (pred_mean=4.406)
[INFO] Skipping submission write (best_thr not available).
--- Variant A: tau=0.05, k=150, beta=0.10 ---


[accum] 2000/21318


[accum] 4000/21318


...

[accum] 14000/21318


[accum] 16000/21318


[accum] 18000/21318


[accum] 20000/21318


[WRITE] submission_knn_softmax_probblend_groupmask.csv thr=0.460 post-cap mean=3.825 min_total=3
Done variants. Elapsed 4.7m


In [66]:
import shutil, pandas as pd, numpy as np
print('=== Set submission to best kNN variant: tau=0.05 prob-blend (min_total=4) ===', flush=True)
src = 'submission_knn_softmax_probblend_tau005.csv'
df = pd.read_csv(src)
mean_cnt = df['attribute_ids'].fillna('').apply(lambda s: 0 if s=='' else len(str(s).split())).mean()
print(f'Source {src} mean count={mean_cnt:.3f}')
shutil.copyfile(src, 'submission.csv')
print('Copied', src, '-> submission.csv')

=== Set submission to best kNN variant: tau=0.05 prob-blend (min_total=4) ===


Source submission_knn_softmax_probblend_tau005.csv mean count=4.395
Copied submission_knn_softmax_probblend_tau005.csv -> submission.csv


In [67]:
import numpy as np, pandas as pd, shutil
from pathlib import Path

print('=== Enforce per-group min-1 on {country,culture,dimension} for tau=0.05 blend (min_total=4) ===', flush=True)
src = Path('submission_knn_softmax_probblend_tau005.csv')
assert src.exists(), 'Missing base submission file submission_knn_softmax_probblend_tau005.csv'

# Load labels and group mapping
labels_df = pd.read_csv('labels.csv')
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
attr_ids_sorted = np.array(sorted(labels_df['attribute_id'].astype(int).unique().tolist()), dtype=np.int32)
attr_to_col = {a:i for i,a in enumerate(attr_ids_sorted)}
idx_to_group = labels_df.set_index('attribute_id').loc[attr_ids_sorted, 'group'].values

# Caps and required groups
caps = {'country':1,'culture':1,'dimension':1,'medium':2,'tags':6,'tag':6}
default_cap = 3
required_groups = ['country','culture','dimension']

# Load model probs for selecting best fills within a group
from __main__ import blend_equal_weight
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
_, _, P_model = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.40)
assert P_model is not None, 'Failed to load P_model'

# Build per-group column indices
group_to_cols = {}
for j, aid in enumerate(attr_ids_sorted):
    g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
    group_to_cols.setdefault(g, []).append(j)

# Load base submission and enforce per-group minimums
sub = pd.read_csv(src)
ids = sub['id'].values
pred_lists = [str(x).strip().split() if isinstance(x, str) else [] for x in sub['attribute_ids'].values]

rows = []; counts = []
for i, (img_id, attrs) in enumerate(zip(ids, pred_lists)):
    chosen_attr_ids = [int(a) for a in attrs if a!='']
    chosen_idx = [attr_to_col[a] for a in chosen_attr_ids if a in attr_to_col]
    used = {}
    for j in chosen_idx:
        g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
        used[g] = used.get(g, 0) + 1
    chosen_set = set(chosen_idx)
    # Enforce per-group min-1 for each required group if under its cap
    for g in required_groups:
        if used.get(g, 0) >= 1:
            continue
        cols = group_to_cols.get(g, [])
        if not cols:
            continue
        kcap = caps.get(g, default_cap)
        c = used.get(g, 0)
        if c >= kcap:
            continue
        # select top label by model prob within this group not already chosen
        probs_g = P_model[i, cols]
        order = np.argsort(-probs_g)
        picked = False
        for oi in order:
            j = int(cols[int(oi)])
            if j in chosen_set:
                continue
            chosen_idx.append(j); chosen_set.add(j); used[g] = c + 1; picked = True
            break
        # if nothing picked (all already present), skip
    pred_attr = [int(attr_ids_sorted[j]) for j in chosen_idx]
    rows.append({'id': img_id, 'attribute_ids': ' '.join(str(x) for x in pred_attr)})
    counts.append(len(pred_attr))

out_path = 'submission_knn_tau005_minGroup.csv'
pd.DataFrame(rows).to_csv(out_path, index=False)
shutil.copyfile(out_path, 'submission.csv')
print(f'Wrote {out_path} (mean count={np.mean(counts):.3f}) and set as submission.csv')

=== Enforce per-group min-1 on {country,culture,dimension} for tau=0.05 blend (min_total=4) ===


[CARD] Train mean labels/img=4.421, target=4.400, chosen thr=0.502 (pred_mean=4.406)
[INFO] Skipping submission write (best_thr not available).


Wrote submission_knn_tau005_minGroup.csv (mean count=5.808) and set as submission.csv


In [68]:
import numpy as np, pandas as pd, shutil, faiss, time
from pathlib import Path

print('=== kNN tau=0.05 prob-blend with per-group min-1 (country,culture,dimension) + thr sweep to mean~4.44 ===', flush=True)
t0 = time.time()

# Load embeddings and build FAISS (reuse config k=150, tau=0.05, IDF^0.5, |L|^-1)
Xtr = np.load('clip_train_emb.npy').astype(np.float32)
Xte = np.load('clip_test_emb.npy').astype(np.float32)
d = Xtr.shape[1]
index = faiss.IndexFlatIP(d); index.add(Xtr)
k = 150; tau = 0.05; gamma_idf = 0.5
sims, nn_idx = index.search(Xte, k)

# Labels and train annotations
labels_df = pd.read_csv('labels.csv')
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
attr_ids_sorted = np.array(sorted(labels_df['attribute_id'].astype(int).unique().tolist()), dtype=np.int32)
attr_to_col = {a:i for i,a in enumerate(attr_ids_sorted)}
idx_to_group = labels_df.set_index('attribute_id').loc[attr_ids_sorted, 'group'].values
C = len(attr_ids_sorted)
train_df = pd.read_csv('train.csv')
n_train = len(train_df)
lab_lists = []; lab_sizes = np.zeros(n_train, dtype=np.int32); df_counts = np.zeros(C, dtype=np.int32)
for i, s in enumerate(train_df['attribute_ids'].fillna('').astype(str).tolist()):
    if s:
        cols = [attr_to_col[int(x)] for x in s.split() if x!='' and int(x) in attr_to_col]
        lab_lists.append(cols); lab_sizes[i] = len(cols)
        for c in set(cols): df_counts[c] += 1
    else:
        lab_lists.append([]); lab_sizes[i] = 0
idf = np.log(n_train / (df_counts.astype(np.float32) + 1.0))
idf = np.clip(idf, 0.0, None) ** gamma_idf

# Softmax weights
sims_sm = sims / max(tau,1e-6); sims_sm -= sims_sm.max(axis=1, keepdims=True)
w = np.exp(sims_sm); w /= (w.sum(axis=1, keepdims=True) + 1e-12)

# Accumulate kNN probs with |L|^-1 and IDF^0.5 + renorm
ntest = Xte.shape[0]
Pk = np.zeros((ntest, C), dtype=np.float32)
for i in range(ntest):
    wi = w[i]; nn = nn_idx[i]
    for nbr, wj in zip(nn, wi):
        Lj = lab_lists[nbr]
        if not Lj: continue
        add = float(wj) / float(max(lab_sizes[nbr], 1))
        for c in Lj: Pk[i, c] += add
    if (i+1) % 2000 == 0: print(f'[accum] {i+1}/{ntest}', flush=True)
Pk *= idf[None, :]
rs = Pk.sum(axis=1, keepdims=True)
Pk = np.divide(Pk, np.where(rs>0, rs, 1.0), out=np.zeros_like(Pk), where=rs>0)

# Load model probs and prob-blend (beta=0.10)
from __main__ import blend_equal_weight
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
_, _, P_model = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.40)
beta = 0.10
P_final = (1.0 - beta) * P_model + beta * Pk

# Caps, required groups, and per-group min rule
caps = {'country':1,'culture':1,'dimension':1,'medium':2,'tags':6,'tag':6}
default_cap = 3
required_groups = ['country','culture','dimension']
group_to_cols = {}
for j, aid in enumerate(attr_ids_sorted):
    g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
    group_to_cols.setdefault(g, []).append(j)

ids = pd.read_csv('sample_submission.csv')['id'].values

def apply_caps_min_total_and_groupmins(Pt, thr, min_total=4):
    rows = []; counts = []
    for i in range(Pt.shape[0]):
        p = Pt[i]
        cand = np.where(p >= thr)[0]
        if cand.size == 0:
            cand = np.array([int(np.argmax(p))], dtype=np.int64)
        cand_sorted = cand[np.argsort(-p[cand])]
        used, kept = {}, []; kept_set = set()
        # Apply caps on thresholded set
        for j in cand_sorted:
            g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
            kcap = caps.get(g, default_cap); c = used.get(g, 0)
            if c < kcap:
                kept.append(j); kept_set.add(j); used[g] = c + 1
        # Enforce per-group min-1 for required groups using P_model ranking within the group
        for g in required_groups:
            if used.get(g, 0) >= 1: continue
            cols = group_to_cols.get(g, [])
            if not cols: continue
            kcap = caps.get(g, default_cap); c = used.get(g, 0)
            if c >= kcap: continue
            probs_g = P_model[i, cols]
            order = np.argsort(-probs_g)
            for oi in order:
                j = int(cols[int(oi)])
                if j in kept_set: continue
                kept.append(j); kept_set.add(j); used[g] = c + 1
                break
        # Enforce min_total using P_model globally
        if len(kept) < min_total:
            order = np.argsort(-P_model[i])
            for j in order:
                if j in kept_set: continue
                g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
                kcap = caps.get(g, default_cap); c = used.get(g, 0)
                if c < kcap:
                    kept.append(j); kept_set.add(j); used[g] = c + 1
                    if len(kept) >= min_total: break
        if len(kept) == 0:
            kept = [int(np.argmax(p))]
        rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(int(attr_ids_sorted[j])) for j in kept)})
        counts.append(len(kept))
    return rows, float(np.mean(counts))

# Sweep global threshold to hit mean ~4.44 (recall-leaning per expert)
thrs = np.arange(0.460, 0.5401, 0.002)
target_mean = 4.44
best = None; best_rows = None
for t in thrs:
    rows_t, mean_t = apply_caps_min_total_and_groupmins(P_final, t, min_total=4)
    delta = abs(mean_t - target_mean)
    if (best is None) or (delta < best[0]):
        best = (delta, t, mean_t); best_rows = rows_t
    if int((t-0.460)/0.002) % 10 == 0:
        print(f'[SWEEP] thr={t:.3f} post-rule mean={mean_t:.3f}', flush=True)

_, best_thr, best_mean = best
out_path = 'submission_knn_tau005_minGroup_swept.csv'
pd.DataFrame(best_rows).to_csv(out_path, index=False)
shutil.copyfile(out_path, 'submission.csv')
print(f'[WRITE] {out_path} thr={best_thr:.3f} post-rule mean={best_mean:.3f} | Elapsed {(time.time()-t0)/60:.1f}m', flush=True)

=== kNN tau=0.05 prob-blend with per-group min-1 (country,culture,dimension) + thr sweep to mean~4.44 ===


[accum] 2000/21318


[accum] 4000/21318


[accum] 6000/21318


[accum] 8000/21318


...

[SWEEP] thr=0.480 post-rule mean=5.287


[SWEEP] thr=0.502 post-rule mean=5.118


[SWEEP] thr=0.520 post-rule mean=4.997


[SWEEP] thr=0.540 post-rule mean=4.880


[WRITE] submission_knn_tau005_minGroup_swept.csv thr=0.540 post-rule mean=4.880 | Elapsed 2.4m


In [69]:
import numpy as np, pandas as pd, shutil, faiss, time
from pathlib import Path

print('=== Re-sweep: kNN tau=0.05 prob-blend + per-group min-1 with higher thr range to hit mean~4.44 ===', flush=True)
t0 = time.time()

# Recompute P_final (tau=0.05, k=150, IDF^0.5, |L|^-1) to avoid depending on cached variables
Xtr = np.load('clip_train_emb.npy').astype(np.float32)
Xte = np.load('clip_test_emb.npy').astype(np.float32)
d = Xtr.shape[1]
index = faiss.IndexFlatIP(d); index.add(Xtr)
k = 150; tau = 0.05; gamma_idf = 0.5
sims, nn_idx = index.search(Xte, k)

labels_df = pd.read_csv('labels.csv')
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
attr_ids_sorted = np.array(sorted(labels_df['attribute_id'].astype(int).unique().tolist()), dtype=np.int32)
attr_to_col = {a:i for i,a in enumerate(attr_ids_sorted)}
idx_to_group = labels_df.set_index('attribute_id').loc[attr_ids_sorted, 'group'].values
C = len(attr_ids_sorted)
train_df = pd.read_csv('train.csv')
n_train = len(train_df)
lab_lists = []; lab_sizes = np.zeros(n_train, dtype=np.int32); df_counts = np.zeros(C, dtype=np.int32)
for i, s in enumerate(train_df['attribute_ids'].fillna('').astype(str).tolist()):
    if s:
        cols = [attr_to_col[int(x)] for x in s.split() if x!='' and int(x) in attr_to_col]
        lab_lists.append(cols); lab_sizes[i] = len(cols)
        for c in set(cols): df_counts[c] += 1
    else:
        lab_lists.append([]); lab_sizes[i] = 0
idf = np.log(n_train / (df_counts.astype(np.float32) + 1.0))
idf = np.clip(idf, 0.0, None) ** gamma_idf

sims_sm = sims / max(tau,1e-6); sims_sm -= sims_sm.max(axis=1, keepdims=True)
w = np.exp(sims_sm); w /= (w.sum(axis=1, keepdims=True) + 1e-12)

ntest = Xte.shape[0]
Pk = np.zeros((ntest, C), dtype=np.float32)
for i in range(ntest):
    wi = w[i]; nn = nn_idx[i]
    for nbr, wj in zip(nn, wi):
        Lj = lab_lists[nbr]
        if not Lj: continue
        add = float(wj) / float(max(lab_sizes[nbr], 1))
        for c in Lj: Pk[i, c] += add
    if (i+1) % 2000 == 0: print(f'[accum] {i+1}/{ntest}', flush=True)
Pk *= idf[None, :]
rs = Pk.sum(axis=1, keepdims=True)
Pk = np.divide(Pk, np.where(rs>0, rs, 1.0), out=np.zeros_like(Pk), where=rs>0)

from __main__ import blend_equal_weight
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
_, _, P_model = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.40)
beta = 0.10
P_final = (1.0 - beta) * P_model + beta * Pk

# Caps and per-group min rule
caps = {'country':1,'culture':1,'dimension':1,'medium':2,'tags':6,'tag':6}
default_cap = 3
required_groups = ['country','culture','dimension']
group_to_cols = {}
for j, aid in enumerate(attr_ids_sorted):
    g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
    group_to_cols.setdefault(g, []).append(j)

ids = pd.read_csv('sample_submission.csv')['id'].values

def apply_caps_min_total_and_groupmins(Pt, thr, min_total=4):
    rows = []; counts = []
    for i in range(Pt.shape[0]):
        p = Pt[i]
        cand = np.where(p >= thr)[0]
        if cand.size == 0:
            cand = np.array([int(np.argmax(p))], dtype=np.int64)
        cand_sorted = cand[np.argsort(-p[cand])]
        used, kept = {}, []; kept_set = set()
        for j in cand_sorted:
            g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
            kcap = caps.get(g, default_cap); c = used.get(g, 0)
            if c < kcap:
                kept.append(j); kept_set.add(j); used[g] = c + 1
        # per-group min-1 for structural groups
        for g in required_groups:
            if used.get(g, 0) >= 1: continue
            cols = group_to_cols.get(g, [])
            if not cols: continue
            kcap = caps.get(g, default_cap); c = used.get(g, 0)
            if c >= kcap: continue
            probs_g = P_model[i, cols]
            order = np.argsort(-probs_g)
            for oi in order:
                j = int(cols[int(oi)])
                if j in kept_set: continue
                kept.append(j); kept_set.add(j); used[g] = c + 1
                break
        # enforce min_total using model ranking
        if len(kept) < min_total:
            order = np.argsort(-P_model[i])
            for j in order:
                if j in kept_set: continue
                g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
                kcap = caps.get(g, default_cap); c = used.get(g, 0)
                if c < kcap:
                    kept.append(j); kept_set.add(j); used[g] = c + 1
                    if len(kept) >= min_total: break
        if len(kept) == 0:
            kept = [int(np.argmax(p))]
        rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(int(attr_ids_sorted[j])) for j in kept)})
        counts.append(len(kept))
    return rows, float(np.mean(counts))

# Higher threshold sweep to bring mean down near 4.44
thrs = np.arange(0.560, 0.7001, 0.002)
target_mean = 4.44
best = None; best_rows = None
for t in thrs:
    rows_t, mean_t = apply_caps_min_total_and_groupmins(P_final, t, min_total=4)
    delta = abs(mean_t - target_mean)
    if (best is None) or (delta < best[0]):
        best = (delta, t, mean_t); best_rows = rows_t
    if int((t-0.560)/0.002) % 20 == 0:
        print(f'[SWEEP] thr={t:.3f} post-rule mean={mean_t:.3f}', flush=True)

_, best_thr, best_mean = best
out_path = 'submission_knn_tau005_minGroup_swept_hi.csv'
pd.DataFrame(best_rows).to_csv(out_path, index=False)
shutil.copyfile(out_path, 'submission.csv')
print(f'[WRITE] {out_path} thr={best_thr:.3f} post-rule mean={best_mean:.3f} | Elapsed {(time.time()-t0)/60:.1f}m', flush=True)

=== Re-sweep: kNN tau=0.05 prob-blend + per-group min-1 with higher thr range to hit mean~4.44 ===


[accum] 2000/21318


[accum] 4000/21318


[accum] 6000/21318


[accum] 8000/21318


...

[SWEEP] thr=0.560 post-rule mean=4.774


[SWEEP] thr=0.600 post-rule mean=4.606


[SWEEP] thr=0.640 post-rule mean=4.480


[SWEEP] thr=0.680 post-rule mean=4.380


[WRITE] submission_knn_tau005_minGroup_swept_hi.csv thr=0.654 post-rule mean=4.441 | Elapsed 3.8m


In [70]:
import numpy as np, pandas as pd, time, shutil, faiss
from pathlib import Path

print('=== kNN union (restricted): add up to L=3 kNN labels from {country,culture,dimension} with gate=0.08; sweep thr to mean~4.40 ===', flush=True)
t0 = time.time()

# Load CLIP embeddings
Xtr = np.load('clip_train_emb.npy').astype(np.float32)
Xte = np.load('clip_test_emb.npy').astype(np.float32)
d = Xtr.shape[1]
index = faiss.IndexFlatIP(d); index.add(Xtr)

# Params
k = 150; tau = 0.10; eta = 1.0; gamma_idf = 0.5
sims, nn_idx = index.search(Xte, k)
print('kNN search done:', sims.shape, flush=True)

# Labels mapping
labels_df = pd.read_csv('labels.csv')
attr_ids_sorted = np.array(sorted(labels_df['attribute_id'].astype(int).unique().tolist()), dtype=np.int32)
attr_to_col = {a:i for i,a in enumerate(attr_ids_sorted)}
C = len(attr_ids_sorted)
train_df = pd.read_csv('train.csv')
n_train = len(train_df)

# Train label lists and sizes + df counts
lab_lists = []; lab_sizes = np.zeros(n_train, dtype=np.int32); df_counts = np.zeros(C, dtype=np.int32)
for i, s in enumerate(train_df['attribute_ids'].fillna('').astype(str).tolist()):
    if s:
        cols = [attr_to_col[int(x)] for x in s.split() if x!='' and int(x) in attr_to_col]
        lab_lists.append(cols); lab_sizes[i] = len(cols)
        for c in set(cols): df_counts[c] += 1
    else:
        lab_lists.append([]); lab_sizes[i] = 0

# Softmax weights with temperature
sims_sm = sims / max(tau,1e-6)
sims_sm -= sims_sm.max(axis=1, keepdims=True)
w = np.exp(sims_sm); w /= (w.sum(axis=1, keepdims=True) + 1e-12)

# Accumulate probs_knn with per-neighbor |L|^-eta
ntest = Xte.shape[0]
probs_knn = np.zeros((ntest, C), dtype=np.float32)
for i in range(ntest):
    nn = nn_idx[i]; wi = w[i]
    for nbr, wj in zip(nn, wi):
        Lj = lab_lists[nbr]
        if not Lj: continue
        denom = (lab_sizes[nbr] ** eta) if eta > 0 else 1.0
        add = float(wj) / float(max(denom, 1.0))
        for c in Lj: probs_knn[i, c] += add
    if (i+1) % 2000 == 0: print(f'[kNN-accum] {i+1}/{ntest}', flush=True)

# IDF^gamma and renormalize
idf = np.log(n_train / (df_counts.astype(np.float32) + 1.0))
idf = np.clip(idf, 0.0, None) ** gamma_idf
probs_knn *= idf[None, :]
row_sums = probs_knn.sum(axis=1, keepdims=True)
probs_knn = np.divide(probs_knn, np.where(row_sums > 0, row_sums, 1.0), out=np.zeros_like(probs_knn), where=row_sums>0)

# Load model probabilities (b3 2:1)
from __main__ import blend_equal_weight
dirs_b3 = ['out_b3_384_top512','out_b3_448_top512']
_, _, P_model = blend_equal_weight(dirs_b3, weights=[2,1], write_submission=False, out_name='noop.csv', cardinality_target=4.40)
assert P_model is not None and P_model.shape == probs_knn.shape

# Groups and caps
labels_df['group'] = labels_df['attribute_name'].astype(str).str.split('::').str[0].str.lower()
idx_to_group = labels_df.set_index('attribute_id').loc[attr_ids_sorted, 'group'].values
allowed_groups = {'country','culture','dimension'}
caps = {'country':1,'culture':1,'dimension':1,'medium':2,'tags':6,'tag':6}
default_cap = 3
ids = pd.read_csv('sample_submission.csv')['id'].values

def build_union_rows_restricted(thr, L=3, gate=0.08, min_total=3):
    rows = []; counts = []
    for i in range(P_model.shape[0]):
        p = P_model[i]; pk = probs_knn[i]
        cand = np.where(p >= thr)[0]
        if cand.size == 0: cand = np.array([int(np.argmax(p))], dtype=np.int64)
        cand_sorted = cand[np.argsort(-p[cand])]
        used = {}; kept = []; kept_set = set()
        # model thresholded + caps
        for j in cand_sorted:
            g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
            kcap = caps.get(g, default_cap); c = used.get(g, 0)
            if c < kcap: kept.append(j); kept_set.add(j); used[g] = c + 1
        # restricted union: only structural groups
        if L > 0:
            order_knn = np.argsort(-pk)
            added = 0
            for j in order_knn:
                if pk[j] < gate: break
                if j in kept_set: continue
                g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
                if g not in allowed_groups: continue
                kcap = caps.get(g, default_cap); c = used.get(g, 0)
                if c < kcap:
                    kept.append(int(j)); kept_set.add(int(j)); used[g] = c + 1; added += 1
                    if added >= L: break
        # enforce min_total using model ranking
        if len(kept) < min_total:
            order = np.argsort(-p)
            for j in order:
                if j in kept_set: continue
                g = idx_to_group[j] if j < len(idx_to_group) else 'misc'
                kcap = caps.get(g, default_cap); c = used.get(g, 0)
                if c < kcap:
                    kept.append(int(j)); kept_set.add(int(j)); used[g] = c + 1
                    if len(kept) >= min_total: break
        if len(kept) == 0: kept = [int(np.argmax(p))]
        rows.append({'id': ids[i], 'attribute_ids': ' '.join(str(int(attr_ids_sorted[j])) for j in kept)})
        counts.append(len(kept))
    return rows, float(np.mean(counts))

# Sweep thr to target mean ~4.40
thrs = np.arange(0.460, 0.5401, 0.002)
target_mean = 4.40
best = None; best_rows = None
for t in thrs:
    rows_t, mean_t = build_union_rows_restricted(t, L=3, gate=0.08, min_total=3)
    delta = abs(mean_t - target_mean)
    if (best is None) or (delta < best[0]):
        best = (delta, t, mean_t); best_rows = rows_t
    if int((t-0.460)/0.002) % 10 == 0:
        print(f'[SWEEP] thr={t:.3f} post-union mean={mean_t:.3f}', flush=True)

_, best_thr, best_mean = best
out_path = 'submission_union_knn_structL3.csv'
pd.DataFrame(best_rows).to_csv(out_path, index=False)
shutil.copyfile(out_path, 'submission.csv')
print(f'[WRITE] {out_path} thr={best_thr:.3f} post-union mean={best_mean:.3f} | Elapsed {(time.time()-t0)/60:.1f}m', flush=True)

=== kNN union (restricted): add up to L=3 kNN labels from {country,culture,dimension} with gate=0.08; sweep thr to mean~4.40 ===


kNN search done: (21318, 150)


[kNN-accum] 2000/21318


[kNN-accum] 4000/21318


[kNN-accum] 6000/21318


...

[SWEEP] thr=0.480 post-union mean=4.404


[SWEEP] thr=0.502 post-union mean=4.175


[SWEEP] thr=0.520 post-union mean=4.009


[SWEEP] thr=0.540 post-union mean=3.855


[WRITE] submission_union_knn_structL3.csv thr=0.480 post-union mean=4.404 | Elapsed 2.8m
